In [1]:
import sys
import os
sys.path.insert(0,os.path.abspath('..'))
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph,MatchBarrels
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from dask import compute,multiprocessing,delayed
import pathlib
import shutil
import glob
import pandas as pd
import vtk
from scipy.spatial import distance
import numpy as np
from sklearn.decomposition import PCA
from scipy.spatial import distance
from scipy.spatial import distance_matrix

# Create output folders

In [2]:
input_path_spatial_graphs = '/nas1/Data_Mythreya/MotorCortexProject/V11/Registration_Final_Coronal_Ref_Frame/Input_Spatial_Graphs/'
input_path_avg_rf = '/nas1/Data_Mythreya/MotorCortexProject/V11/Registration_Final_Coronal_Ref_Frame/Input_Ref_Frame/'
input_path_layer_borders = input_path_avg_rf+'Layer_Borders/'
#input_path_layer_borders_v2 = input_path_avg_rf+'Layer_Borders_v2/'
manual_neuron_axis_path = '/nas1/Data_Mythreya/MotorCortexProject/V11/Registration_Final_Coronal_Ref_Frame/Manual_Neuron_Axis/'
manual_cell_type_assignment = '/nas1/Data_Mythreya/MotorCortexProject/V11/Registration_Final_Coronal_Ref_Frame/vm1_OPTICS_mastersheet_afterMO.xlsx'
layer_borders = input_path_layer_borders + 'vM1_Borders.csv'

surface_resolution = 100

#ref_barrels = AmiraSpatialGraph(input_path_avg_rf+'Avg_SBF_Without_Projections.am',generic_graph=True,\
#                                barrel_projections_present=False)
ref_pia = Surface(input_path_avg_rf+'Avg_Pia_lhs_delunay.vtk')
ref_wm = Surface(input_path_avg_rf+'Avg_WM_lhs_delunay.vtk')
ref_pia_wm = Surface(polydata=ref_pia.surface)
ref_pia_wm.append(ref_wm.surface)

#coronal_pia = Surface(input_path_avg_rf+'MG48_lhs_pia_open_bottom_voxel_size_50.vtk')
#coronal_wm = Surface(input_path_avg_rf+'MG48_lhs_WM_open_bottom_voxel_size_50.vtk')
#coronal_pia_wm = Surface(polydata=coronal_pia.surface)
#coronal_pia_wm.append(coronal_wm.surface)

# make layer border landmarks
border_names = ['L1','L2_3','L5A','L5B',]
#borders = [0.06,0.24,0.48,0.67,0.88]
borders = [0.085,0.209,0.391,0.779]
#bordes_std = [0,0.10,0.10,0.085,0.12]
bordes_std = [0.021,0.080,0.058,0.088]

In [3]:
# clip the ref surfaces
pia_bounds = ref_pia.surface.GetBounds()
wm_bounds = ref_wm.surface.GetBounds()

ref_pia_bottom_open = ref_pia
ref_wm_bottom_open = ref_wm

ref_pia_bottom_open.clip_surface_at_given_x(0,x_limit=pia_bounds[1]-400)
ref_wm_bottom_open.clip_surface_at_given_x(0,x_limit=wm_bounds[1]-400)

ref_pia_bottom_open.write_surface_mesh(input_path_avg_rf+'Avg_Pia_lhs_delunay_bottom_open.vtk')
ref_wm_bottom_open.write_surface_mesh(input_path_avg_rf+'Avg_WM_lhs_delunay_bottom_open.vtk')

ref_pia_wm_bottom_open = Surface(polydata=ref_pia_bottom_open.surface)
ref_pia_wm_bottom_open.append(ref_wm_bottom_open.surface)

In [4]:
# create output folder structure
HOME = str(pathlib.Path(input_path_spatial_graphs).parent)
output_root = HOME + '/Outputs/'
pathlib.Path((output_root)).mkdir(exist_ok=True)

output_surfaces = output_root + '1_Surfaces/'
pathlib.Path((output_surfaces)).mkdir(exist_ok=True)

output_axis_field = output_root + '1_Axis_Fields/'
pathlib.Path((output_axis_field)).mkdir(exist_ok=True)

output_stats = output_root + '2_Local_Stats/'
pathlib.Path((output_stats)).mkdir(exist_ok=True)

output_neuron_axis = output_root + '2_Local_Neuron_Axis/'
pathlib.Path((output_neuron_axis)).mkdir(exist_ok=True)

output_neuron_axis_manual = output_root + '2_Manual_Neuron_Axis/'
pathlib.Path((output_neuron_axis_manual)).mkdir(exist_ok=True)

#output_alignment_using_barrels = output_root + '3_Aligned_By_Barrels/'
#pathlib.Path((output_alignment_using_barrels)).mkdir(exist_ok=True)

output_alignment_using_surfaces = output_root + '3_Aligned_By_Surfaces/'
pathlib.Path((output_alignment_using_surfaces)).mkdir(exist_ok=True)

output_alignment_final = output_root + '4_Final_Registration/'
pathlib.Path((output_alignment_final)).mkdir(exist_ok=True)

output_alignment_using_neuron_axis_and_nearest_rel_depth = output_alignment_final + 'Aligned_Local_Neuron_Axis_To_Nearest_Axis_Rel_Depth_Preserved/'
pathlib.Path((output_alignment_using_neuron_axis_and_nearest_rel_depth)).mkdir(exist_ok=True)

output_alignment_using_neuron_axis_and_local_avg_rel_depth = output_alignment_final + 'Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Rel_Depth_Preserved/'
pathlib.Path((output_alignment_using_neuron_axis_and_local_avg_rel_depth)).mkdir(exist_ok=True)

output_alignment_using_neuron_axis_and_nearest_pia_soma = output_alignment_final + 'Aligned_Local_Neuron_Axis_To_Nearest_Axis_Pia_Soma_Dist_Preserved/'
pathlib.Path((output_alignment_using_neuron_axis_and_nearest_pia_soma)).mkdir(exist_ok=True)

output_alignment_using_neuron_axis_and_local_Avg_pia_soma = output_alignment_final + 'Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Pia_Soma_Dist_Preserved/'
pathlib.Path((output_alignment_using_neuron_axis_and_local_Avg_pia_soma)).mkdir(exist_ok=True)

output_standardization = output_root + '5_Standardized_Neurons/'
pathlib.Path((output_standardization)).mkdir(exist_ok=True)

output_local_rf = output_root + '6_Local_Ref_Frame/'
pathlib.Path((output_local_rf)).mkdir(exist_ok=True)

output_layerwise_celltypes = output_root + '7_Depthwise_Celltypes/'
pathlib.Path((output_layerwise_celltypes)).mkdir(exist_ok=True)

#for file in glob.glob(manual_neuron_axis_path+'*'):
#    shutil.copyfile(file,output_neuron_axis_manual+os.path.basename(file))

#output_canonical = HOME + '/Canonical_View/'
#pathlib.Path((output_canonical)).mkdir(exist_ok=True)

#output_standardization = output_root + '5_Standardized_Neurons/'
#pathlib.Path((output_standardization)).mkdir(exist_ok=True)

# Helper Functions

In [5]:
# rotation from a to b
def align_a_to_b(a,b):
    u_a = Vectors().get_unit_vec(a[0],a[1])
    u_b = Vectors().get_unit_vec(b[0],b[1])
    v = np.cross(u_a,u_b)
    s = np.linalg.norm(v)
    c = np.dot(u_a,u_b)
    I = np.array([1,0,0,0,1,0,0,0,1]).reshape([3,3])
    #print(I)
    v_x = np.array([0,-v[2],v[1],v[2],0,-v[0],-v[1],v[0],0]).reshape([3,3])
    R = I + v_x + np.dot(v_x,v_x) * (1/(1+c))
    r_list = []
    for i in range(3):
        for j in range(3):
            r_list.append(R[i,j])
    #print(R)
    return R

In [6]:
def convert_to_transformation_matrix(rot_mat,translation):
    
    tr_mat = []
    k = 0

    for i in range(3):
        for j in range(3):
            tr_mat.append(rot_mat[i,j])
            k = k + 1
        tr_mat.append(0)
        k = k+1
    tr_mat.append(-translation[0])
    tr_mat.append(-translation[1])
    tr_mat.append(-translation[2])
    tr_mat.append(1)
    return tr_mat

In [7]:
def write_edge(edge,filename):
    axis_sg = AmiraSpatialGraph(generic_graph=True)
    axis_sg.graph_data.add_edge(edge[0],edge[1])
    axis_sg.write_spatial_graph(filename)

    return axis_sg

    

In [8]:
def resample_edge(edge,num_pts_to_be_inserted):
    resampled_edge = []
    #resample_edge.append(edge[0])
    n = np.array(edge[len(edge)-1]) - np.array(edge[0])
    n_unit = n / np.linalg.norm(n)
    edge_len = Vectors().get_vec_length(edge)
    for i in range(num_pts_to_be_inserted):
        resampled_edge.append(i*n_unit*edge_len/num_pts_to_be_inserted+edge[0])
    return edge

In [9]:
def get_pca_direction(apical_dendrite_pts,res=1,two_d_only=False):
#     resampled_pts = []
#     for edge in apical_dendrite_edges:
#         for i in range(len(edge)-1):
#             length = distance.euclidean(edge[i],edge[i+1])
#             if length > res:
#                 num_pts_to_be_inserted = int(length/res)
#                 resampled_edge = resample_edge(edge,num_pts_to_be_inserted)
#                 for i in range(len(resampled_edge)):
#                     resampled_pts.append(resampled_edge[i])
    
    if two_d_only:
        pca = PCA(n_components=2,)
        pts = np.array(apical_dendrite_pts)[:,0:2]
        transformed_pca = pca.fit_transform(X=pts)
    else:
        pca = PCA(n_components=3,)
        transformed_pca = pca.fit_transform(X=apical_dendrite_pts)
    #print(transformed_pca)
    return pca.components_

In [10]:
def get_wm_intersection_pt(wm,soma_pt,pia_pt):
    wm_pt = []
    for j in range(500,6000,50):
        opposing_pt = Vectors().create_pt_along_vector_at_given_distance(-j,soma_pt,pia_pt)
        #print(opposing_pt)
        #Landmarks(pts=[opposing_pt]).write_landmarks(output_root+'oppsing_pt')
        wm_pt,dist = wm.get_vector_intersection_pt(soma_pt,opposing_pt,extrapolation_len=0)
        #Landmarks(pts=[opposing_pt]).write_landmarks(output_root+'wm_pt')
        #print(wm_pt)
        if len(wm_pt)>0:
            break
    return wm_pt

In [11]:
def get_cell_type_from_df(df,exp_name):
    for i in range(len(df)):
        if exp_name.startswith(df['file_id'][i]):
            return df['celltype_manual'][i]
    

# Create Pia and WM Surfaces

In [ ]:
@delayed
def create_surface(file,surface_resolution='100'):
    sg = AmiraSpatialGraph(filename=file)
    # flip if required
    #txmat = get_axis_flip_tx_mat(sg)
    #print(txmat)
    #sg.apply_transformation(txmat)
    #sg.write_spatial_graph(root_dst+os.path.basename(file))
    #sg = AmiraSpatialGraph(root_dst+os.path.basename(file))
    #sg_clipped = get_fist_x_z_planes_sg(sg,17)
    #sg_clipped.write_spatial_graph(root_dst+os.path.basename(file)+'_clipped.am')
    os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format(file,output_surfaces+os.path.basename(file)[:-3],str(100)))
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
    
    if pia.surface.GetNumberOfCells()>0 and wm.surface.GetNumberOfCells()>0:
        pia.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
        wm.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
        
        os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format(file,output_surfaces+os.path.basename(file)[:-3],str(450)))
        pia_450 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
        pia_450.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open_450.vtk')
        
        wm_450 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
        wm_450.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_450.vtk')
            
    else:
        # create a delunay surface as the traditional way has failed
        pia = Surface(pts=sg.pia.edge_pt_coords)
        wm = Surface(pts=sg.wm.edge_pt_coords)
        pia_hull = pia.create_delunay_surface_3d(return_hull=True,output_filename=output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
        wm_hull = wm.create_delunay_surface_3d(return_hull=True,output_filename=output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
        
        # clip at the bottom
        shutil.copyfile(src=file,dst=output_surfaces+os.path.basename(file)[:-3]+'_barrels.am')
        pia.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
        wm.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
        #wm.create_axis_field(pia,op_sg_name=output_surfaces+os.path.basename(file)[:-3]+'_axis_field.am',flip_normals=False,\
        #                              return_axes=True)

In [ ]:
fn_list_surfs = []
file_names = sorted(glob.glob(input_path_spatial_graphs+'*.am'))
for i in range(len(file_names)):
    #if os.path.basename(file_names[i])=='RA20150804_2_4x-_for_cell_merge_.am_Segmented_corrected_DONE.am':
    fn_list_surfs.append(create_surface(file_names[i]))

compute(fn_list_surfs)

# Generate Axis Field

In [ ]:
# Create axis with the smooth wm (voxel size 450) 
file_names = sorted(glob.glob(input_path_spatial_graphs+'*.am'))
for i in range(len(file_names)):
    file = file_names[i]
    #if os.path.basename(file)[:-3] == 'RA20150804_2_4x-_for_cell_merge_.am_Segmented_corrected_DONE':
    pia_100 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm_100 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    wm_450 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_450.vtk')

    if wm_450.surface.GetNumberOfCells() > 0:
        wm_to_use = wm_450
    else:
        wm_to_use = wm_100
    
    flipped_axes = wm_to_use.create_axis_field(pia_100,op_sg_name=None,flip_normals=True,return_axes=True,\
                                            max_allowed_axis_length=5000,check_source_double_touch=True)
    axes = wm_to_use.create_axis_field(pia_100,op_sg_name=None,flip_normals=False,return_axes=True,\
                                    max_allowed_axis_length=5000,check_source_double_touch=True)
    sg = AmiraSpatialGraph(generic_graph=True)
    for edge in flipped_axes:
        sg.graph_data.add_edge(edge[0],edge[1])
    for edge in axes:
        sg.graph_data.add_edge(edge[0],edge[1])
    #sg.write_spatial_graph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am')

    # trim the axis field as per the pia and wm surfaces of 100 micron voxel resolution
    selected_axis_field_sg,wm_pts,pia_pts,missing_axis_field = pia_100.get_axis_field_within_instersecting_surface_with_trimming\
                                (pia_100.surface,wm_100.surface,sg.graph_data.edge_list,vec_extension_offset=0.75)
    selected_axis_field_sg.write_spatial_graph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am')

        
        

# Stats of input neuron and surfaces

In [ ]:
cols = ['Exp_Name','Pia_Area','WM_Area','Pia_Volume','WM_Volume',\
        'Cortical_Thickness_Mean','Cortical_Thickness_Std','Cortical_Thickness_CoV',\
        'Cortical_Thickness_vS1_Mean','Cortical_Thickness_vS1_Std','Cortical_Thickness_vS1_CoV',]
df_main = pd.DataFrame(columns=cols)

for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    print(file)
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    pia_closed = pia.create_delunay_surface_3d(return_hull=True,make_cube=False)
    wm_closed = wm.create_delunay_surface_3d(return_hull=True,make_cube=False)
    axis_field = AmiraSpatialGraph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am',generic_graph=True)
    
    sg = AmiraSpatialGraph(file)
    vs1_sg = AmiraSpatialGraph(generic_graph=True)
    # create hull around s1 and get axis field for s1
    for pt in sg.barrels.all_rows_graphdata.edge_pt_coords:
        min_edge,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = \
        Vectors().get_nearest_axis_to_pt(pt,axis_field.graph_data.edge_list,axis_validation_distance=3000)
        vs1_sg.graph_data.add_edge(min_edge[0],min_edge[1])
    vs1_sg.write_spatial_graph(output_axis_field+os.path.basename(file)[:-3]+'_vS1_Axis_Field.am')
    
    # find vector lengths for whole surface and vs1 area
    vec_lens,bla1,bla2 = Vectors().get_vec_lengths(axis_field.graph_data.edge_list)
    vec_lens_vs1,bla,blee = Vectors().get_vec_lengths(vs1_sg.graph_data.edge_list)
        
    df = pd.DataFrame(columns=df_main.columns)
    df['Exp_Name'] = [os.path.basename(file)[:-3]]
    df['Pia_Area'] = [pia.get_surface_properties(prop='area')]
    df['WM_Area'] = [wm.get_surface_properties(prop='area')]
    df['Pia_Volume'] = [Surface(polydata=pia_closed).get_surface_properties(prop='volume')]
    df['WM_Volume'] = [Surface(polydata=wm_closed).get_surface_properties(prop='volume')]
    df['Cortical_Thickness_Mean'] = [np.array(vec_lens).mean(axis=0)]
    df['Cortical_Thickness_Std'] = [np.array(vec_lens).std(axis=0)]
    df['Cortical_Thickness_CoV'] = df['Cortical_Thickness_Std'] / df['Cortical_Thickness_Mean'] * 100
    df['Cortical_Thickness_vS1_Mean'] = [np.array(vec_lens_vs1).mean(axis=0)]
    df['Cortical_Thickness_vS1_Std'] = [np.array(vec_lens_vs1).std(axis=0)]
    df['Cortical_Thickness_vS1_CoV'] = df['Cortical_Thickness_vS1_Std'] / df['Cortical_Thickness_vS1_Mean'] * 100
        
    df_main = df_main.append(df)

df_main.to_csv(output_stats+'Surface_Area_and_Thickness.csv')
    

In [ ]:
cols = ['Exp_Name','Apical_Dendrite_Present','WM_Present_Below_Neuron',\
        'Neuron_Depth_from_Pia','Neuron_Height_from_WM','Neuron_Axis_Length']
df_main = pd.DataFrame(columns=cols)
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    #if os.path.basename(file)[:-3] == 'RA20160401_Cell_B_DONE':
    print(os.path.basename(file)[:-3])
    df = pd.DataFrame(columns=df_main.columns)
    df['Exp_Name'] = [os.path.basename(file)[:-3]]
    
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    axis_field = AmiraSpatialGraph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am',generic_graph=True)
    
    sg = AmiraSpatialGraph(file)
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    Landmarks(pts=[soma_center]).write_landmarks(output_neuron_axis+os.path.basename(file)[:-3]+'_Soma_Centroid.landmarksAscii')

    dendrite_pts = []    
    for pt in sg.neuron.dendrite.apical_dendrite.edge_pt_coords:
        dendrite_pts.append(pt)
    for pt in sg.neuron.dendrite.basal_dendrite.edge_pt_coords:
        dendrite_pts.append(pt)   
    dendrite_mean = np.array(dendrite_pts).mean(axis=0)

    # write dendrite for easy visualization
    sg.write_sub_spatial_graph(sg.neuron.dendrite.apical_dendrite,output_neuron_axis+os.path.basename(file)[:-3]+\
                               '_apical_dendrite.am')
    sg.write_sub_spatial_graph(sg.neuron.dendrite.basal_dendrite,output_neuron_axis+os.path.basename(file)[:-3]+\
                               '_basal_dendrite.am')

    # Avg Local Axis 
    nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(axis_field.graph_data.edge_list,[soma_center],800)
    nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
    unit_vecs = []
    for axis in nearby_axes:
        nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
        unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
    nearby_axes_sg.write_spatial_graph(output_neuron_axis+os.path.basename(file)[:-3]+'_axis_field_selected.am')
    avg_uv = np.array(unit_vecs).mean(axis=0)
    end_pt = (avg_uv*6000 + soma_center)
    pia_proj_pt_neuron_using_local_avg_axis,soma_depth_using_local_avg = pia.get_vector_intersection_pt(end_pt,soma_center,extrapolation_len=0)
    write_edge([soma_center,pia_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_axis_local_avg.am')
    
    wm_proj_pt_neuron_using_local_avg_axis = get_wm_intersection_pt(wm,soma_center,pia_proj_pt_neuron_using_local_avg_axis)
    if len(wm_proj_pt_neuron_using_local_avg_axis) > 0:
        write_edge([soma_center,wm_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_axis_local_avg.am')
    else:
        print('no wm intersection found using avg local axis for {}'.format(os.path.basename(file)))
    soma_depth_using_local_avg = Vectors().get_vec_length([soma_center,pia_proj_pt_neuron_using_local_avg_axis])
    # decide which method of depth measurement to use
    # using the apical dendrite axis works best for L5 and below neurons
    # where as local axis is good for L2/3
    print(soma_depth_using_local_avg)

    if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords) > 0:
        df['Apical_Dendrite_Present'] = ['Yes']
        #df = pd.DataFrame(columns=df_main.columns)
        # Get apical dend axis using pca
        pca_components = get_pca_direction(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)
        #Landmarks(pts=resampled_pts).write_landmarks(output_root+'pts')
        end_pt = Vectors().create_pt_along_vector_at_given_distance(6000,[0,0,0],pca_components[0,:])
        #Landmarks(pts=[end_pt+soma_center]).write_landmarks(output_root+'end_pt')
        #print(end_pt)
        angle = Vectors().get_angle_between_vectors([soma_center,pia_proj_pt_neuron_using_local_avg_axis],\
                         [soma_center,end_pt+soma_center],ignore_opposite_direction=False)
        print(angle)
        pia_proj_pt_neuron = []
        if angle < 90:
            # pca direction is correct.. project to pia
            end_pt = Vectors().create_pt_along_vector_at_given_distance(6000,[0,0,0],pca_components[0,:])
            print(end_pt)
            Landmarks(pts=[end_pt+soma_center]).write_landmarks(output_root+'end_pt')
            pia_proj_pt_neuron,soma_depth  = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
            write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_pca_axis.am')
        else:
            # the pca is inverted, so straighten it
            #print('opposite')
            end_pt = Vectors().create_pt_along_vector_at_given_distance(-6000,[0,0,0],pca_components[0,:])
            pia_proj_pt_neuron,soma_depth = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
            write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_pca_axis.am')
        
        wm_proj_pt_neuron = get_wm_intersection_pt(wm,soma_center,pia_proj_pt_neuron)
        if len (wm_proj_pt_neuron) > 0:
            write_edge([soma_center,wm_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_pca_axis.am')
        else:
            print('no wm intersection found using pca for {}'.format(os.path.basename(file)))
    else:
        df['Apical_Dendrite_Present'] = ['No']
    # select the neron axis based on whether the depth of the neuron somata
    # for upper layer cells use avg axis, as they generally dont have enough apical dend
    # otherwise use dend pca based axis
    if soma_depth_using_local_avg < 500 or len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords) == 0\
        or os.path.basename(file)[:-3] == 'RA20160401_Cell_B_DONE' or os.path.basename(file)[:-3]=='RA20180412_2_Cell_B.am_Segmented_corrected_DONE':
        # this is a inverted cell... so do not invert the axis.. lets use the avg axis for this one:
        df['Neuron_Depth_from_Pia'] = [distance.euclidean(soma_center,pia_proj_pt_neuron_using_local_avg_axis)]
        write_edge([soma_center,pia_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_axis.am')
        if len(wm_proj_pt_neuron_using_local_avg_axis) > 0:
            write_edge([soma_center,wm_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_axis.am')
            df['WM_Present_Below_Neuron'] = ['Yes']
            df['Neuron_Height_from_WM'] = [distance.euclidean(soma_center,wm_proj_pt_neuron_using_local_avg_axis)]
            df['Neuron_Axis_Length'] = df['Neuron_Depth_from_Pia'] + df['Neuron_Height_from_WM']
            
    else:
        df['Neuron_Depth_from_Pia'] = [distance.euclidean(soma_center,pia_proj_pt_neuron)]
        write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_axis.am')
        if len(wm_proj_pt_neuron) > 0:
            write_edge([soma_center,wm_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_axis.am')
            df['WM_Present_Below_Neuron'] = ['Yes']
            df['Neuron_Height_from_WM'] = [distance.euclidean(soma_center,wm_proj_pt_neuron)]
            df['Neuron_Axis_Length'] = df['Neuron_Depth_from_Pia'] + df['Neuron_Height_from_WM']
            
    df_main = df_main.append(df)
    
df_main.to_csv(output_stats+'Local_Neuron_Axis.csv')

# Register Using Surfaces

In [ ]:
i = 0
pca_aligned_neurons = AmiraSpatialGraph()
surface_aligned_neurons = AmiraSpatialGraph()
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    print(i, os.path.basename(file))
    i = i + 1
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    pia_wm = pia
    pia_wm.append(wm.surface)
    for alignment_type in ['Using_PCA','Using_Surface']:
        if alignment_type is 'Using_PCA':
            input_path = output_root
            output_path = output_alignment_using_surfaces + 'Using_PCA/'
            pathlib.Path((output_path)).mkdir(exist_ok=True)

            ref_pca_pts = []
            pca_pts = []
            #ref_surf = Surface()

            ref_pca_pts = Alignment().get_pca_for_surf(ref_pia_wm_bottom_open.surface,output_path=None,ref_coronal=True,lhs=True)

            pca_pts = Alignment().get_pca_for_surf(pia_wm.surface,output_path=None,ref_coronal=False,lhs=True)


            icp,txmat = Landmarks(pts=pca_pts).align_landmarks(ref_pca_pts)

            aligner = Alignment()
            aligner.transform_folders(input_path,output_path,os.path.basename(file)[:-3],icp=icp,txmat = txmat,)
            
            if os.path.exists(output_alignment_using_surfaces + 'Using_PCA/2_Local_Neuron_Axis/' + os.path.basename(file)[:-3]+'_apical_dendrite.am'):
                sg_apical = AmiraSpatialGraph(output_alignment_using_surfaces + 'Using_PCA/2_Local_Neuron_Axis/' + os.path.basename(file)[:-3]+'_apical_dendrite.am')
                pca_aligned_neurons.graph_data = pca_aligned_neurons.combine_subgraphs([pca_aligned_neurons.graph_data,sg_apical.graph_data])
        
        else:
            input_path = output_alignment_using_surfaces + 'Using_PCA/'
            output_path = output_alignment_using_surfaces + 'Using_Surfaces/'
            pathlib.Path((output_path)).mkdir(exist_ok=True)
            print(input_path,output_path)

            pia = Surface(input_path+'1_Surfaces/'+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
            wm = Surface(output_surfaces+'1_Surfaces/'+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
            pia_wm = Surface()
            pia_wm.append(pia.surface)
            pia_wm.append(wm.surface)
            
            aligner = Alignment(ref_pia_wm_bottom_open.surface,pia_wm.surface,align_by_pca=False,nr_iterations=10000,)
            txmat = aligner.get_transformation_matrix()
            icp = aligner.get_icp_transform()
            aligner.transform_folders(input_path,output_path,os.path.basename(file)[:-3],icp=icp,txmat = txmat,)

            if os.path.exists(output_alignment_using_surfaces + 'Using_Surfaces/2_Local_Neuron_Axis/' + os.path.basename(file)[:-3]+'_apical_dendrite.am'):
                sg_apical = AmiraSpatialGraph(output_alignment_using_surfaces + 'Using_Surfaces/2_Local_Neuron_Axis/' + os.path.basename(file)[:-3]+'_apical_dendrite.am')
                surface_aligned_neurons.graph_data = surface_aligned_neurons.combine_subgraphs([surface_aligned_neurons.graph_data,sg_apical.graph_data])
    
    
pca_aligned_neurons.write_spatial_graph(output_alignment_using_surfaces + 'Using_PCA/All_Apical.am')
surface_aligned_neurons.write_spatial_graph(output_alignment_using_surfaces + 'Using_Surfaces/All_Apical.am')

In [ ]:
#Pia Only
i = 0
pca_aligned_neurons = AmiraSpatialGraph()
surface_aligned_neurons = AmiraSpatialGraph()
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    print(i, os.path.basename(file))
    i = i + 1
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    pia_wm = pia
    pia_wm.append(wm.surface)
    for alignment_type in ['Using_PCA','Using_Surface']:
        if alignment_type is 'Using_PCA':
            input_path = output_root
            output_path = output_alignment_using_surfaces + 'Using_PCA_Pia_Only/'
            pathlib.Path((output_path)).mkdir(exist_ok=True)

            ref_pca_pts = []
            pca_pts = []
            #ref_surf = Surface()

            ref_pca_pts = Alignment().get_pca_for_surf(ref_pia_bottom_open.surface,output_path=None,ref_coronal=True,lhs=True)

            pca_pts = Alignment().get_pca_for_surf(pia.surface,output_path=None,ref_coronal=False,lhs=True)


            icp,txmat = Landmarks(pts=pca_pts).align_landmarks(ref_pca_pts)

            aligner = Alignment()
            aligner.transform_folders(input_path,output_path,os.path.basename(file)[:-3],icp=icp,txmat = txmat,)
            
            if os.path.exists(output_alignment_using_surfaces + 'Using_PCA_Pia_Only/2_Local_Neuron_Axis/' + os.path.basename(file)[:-3]+'_apical_dendrite.am'):
                sg_apical = AmiraSpatialGraph(output_alignment_using_surfaces + 'Using_PCA_Pia_Only/2_Local_Neuron_Axis/' + os.path.basename(file)[:-3]+'_apical_dendrite.am')
                pca_aligned_neurons.graph_data = pca_aligned_neurons.combine_subgraphs([pca_aligned_neurons.graph_data,sg_apical.graph_data])
        
        else:
            input_path = output_alignment_using_surfaces + 'Using_PCA_Pia_Only/'
            output_path = output_alignment_using_surfaces + 'Using_Surfaces_Pia_Only/'
            pathlib.Path((output_path)).mkdir(exist_ok=True)
            print(input_path,output_path)

            pia = Surface(input_path+'1_Surfaces/'+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
            #wm = Surface(output_surfaces+'1_Surfaces/'+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
            pia_wm = Surface()
            pia_wm.append(pia.surface)
            #pia_wm.append(wm.surface)
            
            aligner = Alignment(ref_pia_bottom_open.surface,pia.surface,align_by_pca=False,nr_iterations=10000,)
            txmat = aligner.get_transformation_matrix()
            icp = aligner.get_icp_transform()
            aligner.transform_folders(input_path,output_path,os.path.basename(file)[:-3],icp=icp,txmat = txmat,)

            if os.path.exists(output_alignment_using_surfaces + 'Using_Surfaces_Pia_Only/2_Local_Neuron_Axis/' + os.path.basename(file)[:-3]+'_apical_dendrite.am'):
                sg_apical = AmiraSpatialGraph(output_alignment_using_surfaces + 'Using_Surfaces_Pia_Only/2_Local_Neuron_Axis/' + os.path.basename(file)[:-3]+'_apical_dendrite.am')
                surface_aligned_neurons.graph_data = surface_aligned_neurons.combine_subgraphs([surface_aligned_neurons.graph_data,sg_apical.graph_data])
    
    
pca_aligned_neurons.write_spatial_graph(output_alignment_using_surfaces + 'Using_PCA_Pia_Only/All_Apical.am')
surface_aligned_neurons.write_spatial_graph(output_alignment_using_surfaces + 'Using_Surfaces_Pia_Only/All_Apical.am')

In [ ]:
# Combine the soma centers   
landmarks = Landmarks()
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    landmarks.append_landmarks(Landmarks(output_alignment_using_surfaces + 'Using_Surfaces/2_Local_Neuron_Axis/' + os.path.basename(file)[:-3]+'_Soma_Centroid.landmarksAscii').pts)
    
landmarks.write_landmarks(output_alignment_using_surfaces + 'Using_Surfaces/' + 'All_Somas.landmarksAscii')

In [ ]:
# combine dendrites and neurons
apical_dend = AmiraSpatialGraph()
basal_dends = AmiraSpatialGraph()
#dends = AmiraSpatialGraph()
#neurons = AmiraSpatialGraph()
somas = AmiraSpatialGraph()
axons = AmiraSpatialGraph()
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    sg = AmiraSpatialGraph(output_alignment_using_surfaces + 'Using_Surfaces/1_Surfaces/' + os.path.basename(file)[:-3]+'_barrels.am')
    #apical_dend.graph_data = apical_dend.combine_subgraphs([apical_dend.graph_data,sg.neuron.dendrite.apical_dendrite])
    basal_dends.graph_data = apical_dend.combine_subgraphs([basal_dends.graph_data,sg.neuron.dendrite.basal_dendrite])
    axons.graph_data = axons.combine_subgraphs([axons.graph_data,sg.neuron.axon])
    somas.graph_data = somas.combine_subgraphs([somas.graph_data,sg.neuron.soma])
    
    #surface_aligned_neurons.graph_data = surface_aligned_neurons.combine_subgraphs([surface_aligned_neurons.graph_data,sg_apical.graph_data])
basal_dends.write_spatial_graph(output_alignment_using_surfaces + 'Using_Surfaces/All_Basal.am')
axons.write_spatial_graph(output_alignment_using_surfaces + 'Using_Surfaces/All_Axon.am')
somas.write_spatial_graph(output_alignment_using_surfaces + 'Using_Surfaces/All_Soma.am')

In [ ]:
input_path = output_alignment_using_surfaces + 'Using_Surfaces/'
output_path_nearest_axis = input_path+'Nearest_Axis/'
pathlib.Path(output_path_nearest_axis).mkdir(exist_ok=True)
avg_axis_field = AmiraSpatialGraph(input_path_avg_rf+'vM1_axis_field_lhs.am',generic_graph=True).graph_data.edge_list
nearest_axis_sg = AmiraSpatialGraph(generic_graph=True)
soma_pia_axes_sg = AmiraSpatialGraph(generic_graph=True)
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    #print(i, os.path.basename(file))
    soma_center = Landmarks(input_path + '2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_Soma_Centroid.landmarksAscii').pts[0]
    neaerst_vec,a,b,c,d = Vectors().get_nearest_axis_to_pt(soma_center,avg_axis_field,axis_validation_distance=5000)
    nearest_axis_sg.graph_data.add_edge(neaerst_vec[0],neaerst_vec[1])
    
    nearest_axis_sg_exp = AmiraSpatialGraph(generic_graph=True)
    nearest_axis_sg_exp.graph_data.add_edge(neaerst_vec[0],neaerst_vec[1])
    nearest_axis_sg_exp.write_spatial_graph(output_path_nearest_axis+os.path.basename(file)[:-3]+'_nearest_axis.am')
    
    soma_pia_axis = AmiraSpatialGraph(input_path+'2_Local_Neuron_Axis/'\
                              +os.path.basename(file)[:-3]+'_soma_pia_axis.am',generic_graph=True).graph_data.edge_list[0]
    soma_pia_axes_sg.graph_data.add_edge(soma_pia_axis[0],soma_pia_axis[1])
nearest_axis_sg.write_spatial_graph(input_path+'Nearest_Axes.am')
soma_pia_axes_sg.write_spatial_graph(input_path+'Original_Soma_Pia_Axes.am')

# Refine Using Neuron Axis And Ref Axis

In [ ]:
def refine_registration_using_neuron_axis(input_path,output_path,preserve_rel_depth=True,use_nearest_axis=True):
    ''' Refine registration after the surface to surface registration is done, by aligning neuron axis with ref axis
        use_nearest_axis: whether to use nearest axis in the ref frame or to use the local avg axis
        preserve_rel_depth:whether to preserve the pia soma distance or the relative soma depth in the axis
    '''
    # read ref pia and wm 
    ref_pia = Surface(input_path_avg_rf+'Avg_Pia_lhs_delunay_bottom_open.vtk')
    ref_wm = Surface(input_path_avg_rf+'Avg_WM_lhs_delunay_bottom_open.vtk')
    
    output_path_nearest_axis = input_path+'/Nearest_Axis/'
    pathlib.Path(output_path_nearest_axis).mkdir(exist_ok=True)
    output_path_interpolated_ref_axis = input_path+'/Interpolated_Reference_Axis/'
    pathlib.Path(output_path_interpolated_ref_axis).mkdir(exist_ok=True)
    
    avg_axis_field = AmiraSpatialGraph(input_path_avg_rf+'vM1_axis_field_lhs.am',generic_graph=True).graph_data.edge_list
    nearest_axis_sg = AmiraSpatialGraph(generic_graph=True)
    for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
        print(os.path.basename(file))
        original_axis_field = AmiraSpatialGraph(input_path+'1_Axis_Fields/'+os.path.basename(file)[:-3]+'_axis_field.am',generic_graph=True).graph_data.edge_list

        soma_center = Landmarks(input_path + '2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_Soma_Centroid.landmarksAscii').pts[0]
        soma_pia_axis = AmiraSpatialGraph(input_path+'2_Local_Neuron_Axis/'\
                                  +os.path.basename(file)[:-3]+'_soma_pia_axis.am',generic_graph=True).graph_data.edge_list[0]
        
        if use_nearest_axis:
            neaerst_vec,a,b,c,d = Vectors().get_nearest_axis_to_pt(soma_center,avg_axis_field,axis_validation_distance=5000)
            nearest_axis_sg.graph_data.add_edge(neaerst_vec[0],neaerst_vec[1])
            ref_vec_len = Vectors().get_vec_length(neaerst_vec) 
            nearest_axis_sg_exp = AmiraSpatialGraph(generic_graph=True)
            nearest_axis_sg_exp.graph_data.add_edge(neaerst_vec[0],neaerst_vec[1])
            nearest_axis_sg_exp.write_spatial_graph(output_path_nearest_axis+os.path.basename(file)[:-3]+'_nearest_axis.am')
        else:
            # find the interpolated axis in the ref frame at this position 
            nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(avg_axis_field,[soma_center],800)
            nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
            unit_vecs = []
            for axis in nearby_axes:
                nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
                unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
            nearby_axes_sg.write_spatial_graph(output_path_interpolated_ref_axis+os.path.basename(file)[:-3]+'_axis_field_selected.am')
            avg_uv = np.array(unit_vecs).mean(axis=0)
            end_pt = (avg_uv*6000 + soma_center)
            #print(end_pt)
            wm_proj_pt_neuron_using_local_avg_axis,dist = ref_wm.get_vector_intersection_pt(end_pt,\
                                                        soma_center,extrapolation_len=10000)
            #print(wm_proj_pt_neuron_using_local_avg_axis)
            if len(wm_proj_pt_neuron_using_local_avg_axis) > 0:
                write_edge([soma_center,wm_proj_pt_neuron_using_local_avg_axis],output_path_interpolated_ref_axis+os.path.basename(file)[:-3]+'_soma_wm_axis_local_avg.am')
                
                pia_proj_pt_neuron_using_local_avg_axis,soma_depth_using_local_avg = ref_pia.get_vector_intersection_pt(wm_proj_pt_neuron_using_local_avg_axis,\
                                                                                            soma_center,extrapolation_len=10000)
                write_edge([soma_center,pia_proj_pt_neuron_using_local_avg_axis],\
                       output_path_interpolated_ref_axis+os.path.basename(file)[:-3]+'_soma_pia_axis_local_avg.am')
            
                neaerst_vec = [wm_proj_pt_neuron_using_local_avg_axis,pia_proj_pt_neuron_using_local_avg_axis]
            else:
                print('no wm intersection found using avg local axis for {}'.format(os.path.basename(file)))
            
            
        if preserve_rel_depth:
            # need to find the pia wm dist in the original neuron axis
            if os.path.exists(input_path+'2_Local_Neuron_Axis/'+os.path.basename(file)[:-3]+'_soma_wm_axis.am'):
                soma_wm_axis = AmiraSpatialGraph(input_path+'2_Local_Neuron_Axis/'\
                                          +os.path.basename(file)[:-3]+'_soma_wm_axis.am',generic_graph=True).graph_data.edge_list[0]
                original_vec_length = Vectors().get_vec_length(soma_pia_axis) + Vectors().get_vec_length(soma_wm_axis)
            else:
                # wm intersection not found.. try to find the nearest axis length in the original
                neaerst_vec_original,a,b,c,d = Vectors().get_nearest_axis_to_pt(soma_center,original_axis_field,axis_validation_distance=5000)
                original_vec_length = Vectors().get_vec_length(neaerst_vec_original) 


            # Get soma and pia pts on the ref axis based on the scaled soma pia depth
            len_ratio = Vectors().get_vec_length(neaerst_vec)/original_vec_length
            new_soma_pia_dist = len_ratio * Vectors().get_vec_length(soma_pia_axis) 
            soma_pt_on_ref_axis = Vectors().create_pt_along_vector_at_given_distance(new_soma_pia_dist,\
                                            neaerst_vec[1],neaerst_vec[0])
            pia_pt_on_ref_axis = Vectors().create_pt_along_vector_at_given_distance(Vectors().get_vec_length(soma_pia_axis),\
                                            soma_pt_on_ref_axis,neaerst_vec[1])

            landmarks = Landmarks(pts=soma_pia_axis)
            icp,txmat = landmarks.align_landmarks([soma_pt_on_ref_axis,pia_pt_on_ref_axis])
        else:
            # conserve the pia to soma dist only
            soma_pt_on_ref_axis = Vectors().create_pt_along_vector_at_given_distance(Vectors().get_vec_length(soma_pia_axis),\
                                            neaerst_vec[1],neaerst_vec[0])
            
            landmarks = Landmarks(pts=soma_pia_axis)
            icp,txmat = landmarks.align_landmarks([soma_pt_on_ref_axis,neaerst_vec[1]])
        
        #print(txmat)

        aligner2 = Alignment() 
        aligner2.transform_folders(input_path,output_path,os.path.basename(file)[:-3],txmat=txmat,icp=icp)
        
        

In [ ]:
# register using 4 setups 
# preserving relative depth or pia soma dist
# using nearest axis in the ref frame or local avg axis 
refine_registration_using_neuron_axis(output_alignment_using_surfaces+'Using_Surfaces/',output_alignment_using_neuron_axis_and_nearest_rel_depth,\
                                      use_nearest_axis=True,preserve_rel_depth=True)

refine_registration_using_neuron_axis(output_alignment_using_surfaces+'Using_Surfaces/',output_alignment_using_neuron_axis_and_nearest_pia_soma,\
                                      use_nearest_axis=True,preserve_rel_depth=False)

refine_registration_using_neuron_axis(output_alignment_using_surfaces+'Using_Surfaces/',output_alignment_using_neuron_axis_and_local_avg_rel_depth,\
                                      use_nearest_axis=False,preserve_rel_depth=True)

refine_registration_using_neuron_axis(output_alignment_using_surfaces+'Using_Surfaces/',output_alignment_using_neuron_axis_and_local_Avg_pia_soma,\
                                      use_nearest_axis=False,preserve_rel_depth=False)

In [ ]:
# pull back the sticking neurons


In [ ]:
for input_path in [output_alignment_using_neuron_axis_and_nearest_rel_depth,output_alignment_using_neuron_axis_and_nearest_pia_soma,\
                   output_alignment_using_neuron_axis_and_local_avg_rel_depth,output_alignment_using_neuron_axis_and_local_Avg_pia_soma]:
    apical = AmiraSpatialGraph()
    basal = AmiraSpatialGraph()
    dend = AmiraSpatialGraph()
    #axon = AmiraSpatialGraph()
    neuron = AmiraSpatialGraph()
    #input_path = output_alignment_using_neuron_axis
    for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
        sg = AmiraSpatialGraph(input_path+'1_Surfaces/'+os.path.basename(file)[:-3]+'_barrels.am')
        neuron.graph_data = neuron.combine_subgraphs([neuron.graph_data,sg.neuron.all_neurites_subgraphdata])
        if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)>0:
            apical.graph_data = apical.combine_subgraphs([apical.graph_data,sg.neuron.dendrite.apical_dendrite])
        if len(sg.neuron.dendrite.basal_dendrite.edge_pt_coords)>0:
            basal.graph_data = basal.combine_subgraphs([basal.graph_data,sg.neuron.dendrite.basal_dendrite])

    neuron.write_spatial_graph(input_path+'All_Neurons_Registered.am')
    apical.write_spatial_graph(input_path+'All_Apical_Dendrites_Registered.am')
    basal.write_spatial_graph(input_path+'All_Basal_Dendrites_Registered.am')

In [ ]:
# 

# Sort Regiseted Neurons as per manual cell type assignment

In [ ]:
# write individual neurons into their cell type folder
# Also write the cell layer as per registered depth

#input_path = output_alignment_using_neuron_axis + '1_Surfaces/'
cell_types_df = pd.read_excel(manual_cell_type_assignment,sheetname='celltype_MO')
#output_path_cell_type = output_alignment_using_neuron_axis+'/Manual_Cell_Type_Assignment/'
#pathlib.Path(output_path_cell_type).mkdir(exist_ok=True)

for input_path in [output_alignment_using_neuron_axis_and_nearest_rel_depth,output_alignment_using_neuron_axis_and_nearest_pia_soma,\
                   output_alignment_using_neuron_axis_and_local_avg_rel_depth,output_alignment_using_neuron_axis_and_local_Avg_pia_soma]:
    
    output_path_cell_type = input_path+'/Manual_Cell_Type_Assignment/'
    pathlib.Path(output_path_cell_type).mkdir(exist_ok=True)

    for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
        #for i in range(len(cell_types_df)):
        exp_name = os.path.basename(file)[:-3]
        sg = AmiraSpatialGraph(input_path+'1_Surfaces/'+exp_name+'_barrels.am')
        cell_type = get_cell_type_from_df(cell_types_df,exp_name)
        print(exp_name,cell_type)
        
        outputpath = output_path_cell_type+'/{}/'.format(cell_type)
        pathlib.Path(outputpath).mkdir(exist_ok=True)
        apical = AmiraSpatialGraph()
        basal = AmiraSpatialGraph()
        dend = AmiraSpatialGraph()
        #axon = AmiraSpatialGraph()
        neuron = AmiraSpatialGraph()

        neuron.graph_data = neuron.combine_subgraphs([neuron.graph_data,sg.neuron.all_neurites_subgraphdata])
        if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)>0:
            apical.graph_data = apical.combine_subgraphs([apical.graph_data,sg.neuron.dendrite.apical_dendrite])
            dend.graph_data = dend.combine_subgraphs([dend.graph_data,sg.neuron.dendrite.apical_dendrite])
        if len(sg.neuron.dendrite.basal_dendrite.edge_pt_coords)>0:
            basal.graph_data = basal.combine_subgraphs([basal.graph_data,sg.neuron.dendrite.basal_dendrite])
            dend.graph_data = dend.combine_subgraphs([dend.graph_data,sg.neuron.dendrite.basal_dendrite])

        neuron.write_spatial_graph(outputpath+exp_name+'_Neuron.am')
        apical.write_spatial_graph(outputpath+exp_name+'_Apical.am')
        basal.write_spatial_graph(outputpath+exp_name+'_Basal.am')
        dend.write_spatial_graph(outputpath+exp_name+'_Dend.am')
        
        

In [ ]:
# combine all type specific neurons into each cell type class
#input_folder = output_alignment_using_neuron_axis+'/Manual_Cell_Type_Assignment/'
for input_folder in [output_alignment_using_neuron_axis_and_nearest_rel_depth,output_alignment_using_neuron_axis_and_nearest_pia_soma,\
                   output_alignment_using_neuron_axis_and_local_avg_rel_depth,output_alignment_using_neuron_axis_and_local_Avg_pia_soma]:
    
    for folder in glob.glob(input_folder+'/Manual_Cell_Type_Assignment/*'):
        print(folder)
        
        # combine all neurons
        apical = AmiraSpatialGraph()
        basal = AmiraSpatialGraph()
        dend = AmiraSpatialGraph()
        neuron = AmiraSpatialGraph()
        for file in (glob.glob(folder+'/*_Apical.am')):
            sg_apical = AmiraSpatialGraph(file)
            apical.graph_data = apical.combine_subgraphs([apical.graph_data,sg_apical.graph_data])
        for file in (glob.glob(folder+'/*_Basal.am')):
            sg_basal = AmiraSpatialGraph(file)
            basal.graph_data = basal.combine_subgraphs([basal.graph_data,sg_basal.graph_data])
        for file in (glob.glob(folder+'/*_Dend.am')):
            sg_dend = AmiraSpatialGraph(file)
            dend.graph_data = dend.combine_subgraphs([dend.graph_data,sg_dend.graph_data])
        for file in (glob.glob(folder+'/*_Neuron.am')):
            sg_neuron = AmiraSpatialGraph(file)
            neuron.graph_data = neuron.combine_subgraphs([neuron.graph_data,sg_neuron.graph_data])

        neuron.write_spatial_graph(folder+'/All_Neuron_{}.am'.format(os.path.basename(folder)))
        apical.write_spatial_graph(folder+'/All_Apical_{}.am'.format(os.path.basename(folder)))
        basal.write_spatial_graph(folder+'/All_Basal_{}.am'.format(os.path.basename(folder)))
        dend.write_spatial_graph(folder+'/All_Dend_{}.am'.format(os.path.basename(folder)))
    
    

# Standardize and Write Galleries

In [ ]:
def draw_horzontal_lines(a,b,op_filename):
    l1_line_v1 = AmiraSpatialGraph(generic_graph=True)
    l1_line_v1.graph_data.add_edge(a,b)
    l1_line_v1.write_spatial_graph(op_filename)

In [ ]:
def draw_horzontal_lines_given_depth_profile(ref_len,layer_borders,layer_borders_std,layer_border_names,op_path):
    pia_line = AmiraSpatialGraph(generic_graph=True)
    pia_line.graph_data.add_edge([0,0,-0],[20000,0,-0])
    pia_line.write_spatial_graph(op_path+'Pia_Line_v2.am')
    i = 0
    for border in layer_borders:
        
        line = AmiraSpatialGraph(generic_graph=True)
        line_err_band = AmiraSpatialGraph(generic_graph=True)
        #line_ustd = AmiraSpatialGraph(generic_graph=True)
        #line_lstd = AmiraSpatialGraph(generic_graph=True)
        line.graph_data.add_edge([0,0,-(ref_len*border)],[20000,0,-(ref_len*border)])
        line_err_band.graph_data.add_edge([0,0,-(ref_len*border - ref_len*layer_borders_std[i] )],[20000,0,-(ref_len*border - ref_len*layer_borders_std[i])])
        line_err_band.graph_data.add_edge([0,0,-(ref_len*border + ref_len*layer_borders_std[i])],[20000,0,-(ref_len*border + ref_len*layer_borders_std[i])])
        line.write_spatial_graph( op_path + layer_border_names[i] + '_Line_v2.am' )
        line_err_band.write_spatial_graph( op_path + layer_border_names[i] + '_Error_Band_v2.am' )
        #line_ustd.write_spatial_graph( op_path + layer_border_names[i] + '_Line_upper_band_v2.am' )
        #line_lstd.write_spatial_graph( op_path + layer_border_names[i] + '_Line_lower_band_v2.am' )
        
        # write the boundary error band as a vtk file
        Surface(pts=line_err_band.graph_data.edge_pt_coords).create_delunay_surface_3d(make_cube=True,output_filename = op_path + layer_border_names[i] + '_Error_Band.vtk' )
        
        i = i +1
    wm_line = AmiraSpatialGraph(generic_graph=True)
    wm_line.graph_data.add_edge([0,0,-ref_len],[20000,0,-ref_len])
    wm_line.write_spatial_graph(op_path+'WM_Line_v2.am')

In [ ]:
# Find the central axis as the centroid of the registered somas
borders_folder = output_standardization+'/Central_Axis_And_Borders/'
pathlib.Path(borders_folder).mkdir(exist_ok=True)
registered_soma_centers = []
for file in sorted(glob.glob(output_alignment_using_neuron_axis_and_local_Avg_pia_soma+'/2_Local_Neuron_Axis/'+'*_Soma_Centroid.landmarksAscii')):
    registered_soma_centers.append(Landmarks(file).pts[0])
centroid = np.array(registered_soma_centers).mean(axis=0)
Landmarks(pts=[centroid]).write_landmarks(borders_folder+'somas_centroid.landmarksAscii')
Landmarks(pts=registered_soma_centers).write_landmarks(borders_folder+'registered_soma_centers.landmarksAscii')

# generate a central axis at the somas centroid in the avg rf
ref_pia = Surface(input_path_avg_rf+'Avg_Pia_lhs_delunay.vtk')
ref_wm = Surface(input_path_avg_rf+'Avg_WM_lhs_delunay.vtk')

avg_axis_field = AmiraSpatialGraph(input_path_avg_rf+'vM1_axis_field_lhs.am',generic_graph=True).graph_data.edge_list
nearest_axis_sg = AmiraSpatialGraph(generic_graph=True)
nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(avg_axis_field,[centroid],500)
nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
unit_vecs = []
for axis in nearby_axes:
    nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
    unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
nearby_axes_sg.write_spatial_graph(borders_folder+'nearby_axes.am')
avg_uv = np.array(unit_vecs).mean(axis=0)
end_pt = (avg_uv*6000 + centroid)
#print(end_pt)
wm_proj_pt_neuron_using_local_avg_axis,dist = ref_wm.get_vector_intersection_pt(end_pt,\
                                            centroid,extrapolation_len=10000)
pia_proj_pt_neuron_using_local_avg_axis,soma_depth_using_local_avg = ref_pia.get_vector_intersection_pt(wm_proj_pt_neuron_using_local_avg_axis,\
                                                                                centroid,extrapolation_len=10000)
central_axis_sg = AmiraSpatialGraph(generic_graph=True)
central_axis_sg.graph_data.add_edge(wm_proj_pt_neuron_using_local_avg_axis,pia_proj_pt_neuron_using_local_avg_axis)
central_axis_sg.write_spatial_graph(borders_folder+'central_axis.am')
central_axis = [wm_proj_pt_neuron_using_local_avg_axis,pia_proj_pt_neuron_using_local_avg_axis]

# make layer border landmarks
border_names = ['L1','L2_3','L5A','L5B',]
#borders = [0.06,0.24,0.48,0.67,0.88]
borders = [0.085,0.209,0.391,0.779]
#bordes_std = [0,0.10,0.10,0.085,0.12]
bordes_std = [0.021,0.080,0.058,0.088]
draw_horzontal_lines_given_depth_profile(Vectors().get_vec_length(central_axis),borders,bordes_std,border_names,borders_folder)


# read layer border csv
# lb_df = pd.read_csv(layer_borders_v1)
# l1_dist = Vectors().get_vec_length(central_axis_sg.graph_data.edge_list[0])*(abs(lb_df['L1_Per_Depth'][6])/100)
# l2_3_dist = Vectors().get_vec_length(central_axis_sg.graph_data.edge_list[0])*abs(lb_df['L2/3_Per_Depth'][6])/100
# l5A_dist = Vectors().get_vec_length(central_axis_sg.graph_data.edge_list[0])*abs(lb_df['L5A_Per_Depth'][6])/100
# l5B_dist = Vectors().get_vec_length(central_axis_sg.graph_data.edge_list[0])*abs(lb_df['L5B_Per_Depth'][6])/100
# l6A_dist = Vectors().get_vec_length(central_axis_sg.graph_data.edge_list[0])*abs(lb_df['L6A_Per_Depth'][6])/100

# l1_pt = Vectors().create_pt_along_vector_at_given_distance(l1_dist,pia_proj_pt_neuron_using_local_avg_axis,wm_proj_pt_neuron_using_local_avg_axis)
# l2_3_pt = Vectors().create_pt_along_vector_at_given_distance(l2_3_dist,pia_proj_pt_neuron_using_local_avg_axis,wm_proj_pt_neuron_using_local_avg_axis)
# l5A_pt = Vectors().create_pt_along_vector_at_given_distance(l5A_dist,pia_proj_pt_neuron_using_local_avg_axis,wm_proj_pt_neuron_using_local_avg_axis)
# l5B_pt = Vectors().create_pt_along_vector_at_given_distance(l5B_dist,pia_proj_pt_neuron_using_local_avg_axis,wm_proj_pt_neuron_using_local_avg_axis)
# l6A_pt = Vectors().create_pt_along_vector_at_given_distance(l6A_dist,pia_proj_pt_neuron_using_local_avg_axis,wm_proj_pt_neuron_using_local_avg_axis)

# Landmarks(pts=[pia_proj_pt_neuron_using_local_avg_axis]).write_landmarks(borders_folder+'Pia_pt_v1.landmarkAscii')
# Landmarks(pts=[l1_pt]).write_landmarks(borders_folder+'L1_pt_v1.landmarkAscii')
# Landmarks(pts=[l2_3_pt]).write_landmarks(borders_folder+'L2_3_pt_v1.landmarkAscii')
# Landmarks(pts=[l5A_pt]).write_landmarks(borders_folder+'L5A_pt_v1.landmarkAscii')
# Landmarks(pts=[l5B_pt]).write_landmarks(borders_folder+'L5B_pt_v1.landmarkAscii')
# Landmarks(pts=[l6A_pt]).write_landmarks(borders_folder+'L6A_pt_v1.landmarkAscii')
# Landmarks(pts=[wm_proj_pt_neuron_using_local_avg_axis]).write_landmarks(borders_folder+'WM_pt_v1.landmarkAscii')

# # Verticalize all these things

# landmarks = Landmarks(pts=[central_axis[1],central_axis[0]])
# icp,txmat = landmarks.align_landmarks([[0,0,0],[0,0,-Vectors().get_vec_length(central_axis)]])
# borders_folder_vertical = borders_folder+'/Vertical/'
# pathlib.Path(borders_folder_vertical).mkdir(exist_ok=True)
# Alignment().transform_folder(borders_folder,borders_folder_vertical,txmat=txmat,icp=icp)

# # draw vertical border lines
# draw_horzontal_lines([0,0,-0],[20000,0,-0],borders_folder_vertical+'Pia_Line_v1.am')
# draw_horzontal_lines([0,0,-l1_dist],[20000,0,-l1_dist],borders_folder_vertical+'L1_Line_v1.am')
# draw_horzontal_lines([0,0,-l2_3_dist],[20000,0,-l2_3_dist],borders_folder_vertical+'L2_3_Line_v1.am')
# draw_horzontal_lines([0,0,-l5A_dist],[20000,0,-l5A_dist],borders_folder_vertical+'L5A_Line_v1.am')
# draw_horzontal_lines([0,0,-l5B_dist],[20000,0,-l5B_dist],borders_folder_vertical+'L5B_Line_v1.am')
# draw_horzontal_lines([0,0,-l6A_dist],[20000,0,-l6A_dist],borders_folder_vertical+'L6A_Line_v1.am')
# draw_horzontal_lines([0,0,-Vectors().get_vec_length(central_axis)],[20000,0,-Vectors().get_vec_length(central_axis)],borders_folder_vertical+'WM_Line_v1.am')


In [ ]:
# read the cell type specific neurons, verticalize them and organize into galleries
standardized_no_scaling_neurons_path = output_standardization + '/Standardized_No_Scaling/' 
standardized_uniform_scaling_neurons_path = output_standardization  + '/Standardized_Uniform_Scaling/' 
#standardized_layerwise_scaling_neurons_path = output_standardization  + '/Standardized_Layerwise_Scaling/' 
pathlib.Path(standardized_no_scaling_neurons_path).mkdir(exist_ok=True)
pathlib.Path(standardized_uniform_scaling_neurons_path).mkdir(exist_ok=True)
#pathlib.Path(standardized_layerwise_scaling_neurons_path).mkdir(exist_ok=True)
central_axis = AmiraSpatialGraph(output_standardization+'Central_Axis_And_Borders/central_axis.am',generic_graph=True).graph_data.edge_list[0]
for input_folder in [output_alignment_using_neuron_axis_and_local_avg_rel_depth,\
                     output_alignment_using_neuron_axis_and_local_Avg_pia_soma]:
    
    #print(os.path.basename(input_folder[:-1]))
    standardized_no_scaling_neurons_path = output_standardization + '/Standardized_No_Scaling/' + \
                                                os.path.basename(input_folder[:-1]) + '/'
    standardized_uniform_scaling_neurons_path = output_standardization  + '/Standardized_Uniform_Scaling/' + \
                                                    os.path.basename(input_folder[:-1])+ '/'
    #standardized_layerwise_scaling_neurons_path = output_standardization  + '/Standardized_Layerwise_Scaling/' + \
    #                                                os.path.basename(input_folder[:-1])+ '/'
    pathlib.Path(standardized_no_scaling_neurons_path).mkdir(exist_ok=True)
    pathlib.Path(standardized_uniform_scaling_neurons_path).mkdir(exist_ok=True)
    #pathlib.Path(standardized_layerwise_scaling_neurons_path).mkdir(exist_ok=True)
    
    for folder in sorted(glob.glob(input_folder+'/Manual_Cell_Type_Assignment/*')):
        print(folder)
        gallery_no_scaling_sg = AmiraSpatialGraph()
        gallery_uniform_scaling_sg = AmiraSpatialGraph()
        #gallery_layerwise_scaling_sg = AmiraSpatialGraph()
        gallery_all_sg = AmiraSpatialGraph()
        i = 500
        
        for file in sorted(glob.glob(folder+'/*_Neuron.am')):
            #i = i + 1
            exp_name = os.path.basename(file)[:-10]
            # first verticalize for gallery view
            sg_neuron = AmiraSpatialGraph(file)
            sg_dend = AmiraSpatialGraph(file[:-10]+'_Dend.am')
            # read the soma pia axis and soma center
            if input_folder.find('Nearest')>0:
                soma_center = Landmarks(input_folder+'/2_Local_Neuron_Axis/'+exp_name+\
                                        '_Soma_Centroid.landmarksAscii').pts[0]
            
                neuron_axis = AmiraSpatialGraph(output_alignment_using_surfaces+'Using_Surfaces/Nearest_Axis/'+exp_name+'_nearest_axis.am',\
                                             generic_graph=True).graph_data.edge_list[0]
            else:
                soma_center = Landmarks(input_folder+'/2_Local_Neuron_Axis/'+exp_name+\
                                        '_Soma_Centroid.landmarksAscii').pts[0]
            
                soma_pia_axis = AmiraSpatialGraph(output_alignment_using_surfaces+'Using_Surfaces/Interpolated_Reference_Axis/'+\
                                                exp_name+'_soma_pia_axis_local_avg.am',\
                                             generic_graph=True).graph_data.edge_list[0]
                soma_wm_axis = AmiraSpatialGraph(output_alignment_using_surfaces+'Using_Surfaces/Interpolated_Reference_Axis/'+\
                                                exp_name+'_soma_wm_axis_local_avg.am',\
                                             generic_graph=True).graph_data.edge_list[0]
                neuron_axis = [soma_wm_axis[1],soma_pia_axis[1]]
            
            soma_pia_axis = [soma_center, neuron_axis[1]]
            
            # align neuron to the negative z axis
            rot_mat = align_a_to_b(soma_pia_axis,[[0,0,0],[0,0,1]])
            translation = neuron_axis[1]

            sg_neuron.apply_rotation(rot_mat,translation=translation)
            sg_neuron.write_spatial_graph(standardized_no_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            sg_dend.apply_rotation(rot_mat,translation=translation)
            sg_dend.write_spatial_graph(standardized_no_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            
            # space them so as to visualize better
            sg_dend_tr = AmiraSpatialGraph(standardized_no_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            sg_neuron_tr = AmiraSpatialGraph(standardized_no_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            
            sg_dend_tr.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i,0,0])
            i = i + 400
            gallery_no_scaling_sg.graph_data = gallery_no_scaling_sg.combine_subgraphs([gallery_no_scaling_sg.graph_data,\
                                                                                    sg_dend_tr.graph_data])
            
            # scale uniformely to the central axis
            scaling_factor = Vectors().get_vec_length(central_axis) / Vectors().get_vec_length(neuron_axis)
            sg_dend.graph_data.apply_scaling([scaling_factor,scaling_factor,scaling_factor])
            sg_neuron.graph_data.apply_scaling([scaling_factor,scaling_factor,scaling_factor])
            sg_dend.write_spatial_graph(standardized_uniform_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            sg_neuron.write_spatial_graph(standardized_uniform_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            
            sg_dend_scaling = AmiraSpatialGraph(standardized_uniform_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            sg_neuron_scaling = AmiraSpatialGraph(standardized_uniform_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            
            sg_dend_scaling.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i,0,0])
            i = i + 600
            gallery_uniform_scaling_sg.graph_data = gallery_uniform_scaling_sg.combine_subgraphs([gallery_uniform_scaling_sg.graph_data,\
                                                                                    sg_dend_scaling.graph_data])
            
            
            # scale uniformely to the central axis
            scaling_factor = Vectors().get_vec_length(central_axis) / Vectors().get_vec_length(neuron_axis)
            sg_dend.graph_data.apply_scaling([scaling_factor,scaling_factor,scaling_factor])
            sg_neuron.graph_data.apply_scaling([scaling_factor,scaling_factor,scaling_factor])
            sg_dend.write_spatial_graph(standardized_uniform_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            sg_neuron.write_spatial_graph(standardized_uniform_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            
            sg_dend_scaling = AmiraSpatialGraph(standardized_uniform_scaling_neurons_path+exp_name+'_Dend_{}.am'.format(os.path.basename(folder)))
            sg_neuron_scaling = AmiraSpatialGraph(standardized_uniform_scaling_neurons_path+exp_name+'_Neuron_{}.am'.format(os.path.basename(folder)))
            
            sg_dend_scaling.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i,0,0])
            i = i + 600
            gallery_uniform_scaling_sg.graph_data = gallery_uniform_scaling_sg.combine_subgraphs([gallery_uniform_scaling_sg.graph_data,\
                                                                                    sg_dend_scaling.graph_data])
            
        
        gallery_no_scaling_sg.write_spatial_graph(standardized_no_scaling_neurons_path+os.path.basename(folder)+'_No_Scaling_Gallery.am')
        gallery_uniform_scaling_sg.write_spatial_graph(standardized_uniform_scaling_neurons_path+os.path.basename(folder)+'_Uniform_Scaling_Gallery.am')
        #gallery_uniform_scaling_sg.write_spatial_graph(standardized_uniform_scaling_neurons_path+os.path.basename(folder)+'_Uniform_Scaling_Gallery.am')
        

In [ ]:
# Find soma pia depth
input_sp_path = output_standardization + 'Standardized_No_Scaling/' + os.path.basename(output_alignment_using_neuron_axis_and_local_Avg_pia_soma[:-1])
#len(glob.glob(input_sp_path+'/*_Neuron_*.am'))
soma_depths_csv = pd.DataFrame(columns=['Exp_Name','Soma_Depth_From_Pia'])
for file in sorted(glob.glob(input_sp_path+'/*_Neuron_*.am')):
    sg = AmiraSpatialGraph(file)
    soma_centroid = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    Landmarks(pts=[soma_centroid]).write_landmarks(input_sp_path+'/'+os.path.basename(file)+'_Soma_Centroid.landmarksAscii')
    df=pd.DataFrame()
    df['Exp_Name'] = [os.path.basename(file)[:-14]]
    df['Soma_Depth_From_Pia'] = soma_centroid[2]
    soma_depths_csv = soma_depths_csv.append(df)
    #print(os.path.basename(file),soma_centroid,)
    
soma_depths_csv.to_csv(input_sp_path+'/Soma_Depth_From_Pia.csv')

# Transform to Local vM1 Reference Frame

In [ ]:
pca0 = AmiraSpatialGraph(input_path_avg_rf + 'Avg_vM1_lhs_PCA0_Trimmed.am',generic_graph=True).graph_data.edge_list[0]
pca1 = AmiraSpatialGraph(input_path_avg_rf + 'Avg_vM1_lhs_PCA1_Trimmed.am',generic_graph=True).graph_data.edge_list[0]
pca2 = AmiraSpatialGraph(input_path_avg_rf + 'Avg_vM1_lhs_PCA2_Trimmed.am',generic_graph=True).graph_data.edge_list[0]

pts = []
pts.append(pca0[0])
pts.append(Vectors().create_pt_along_vector_at_given_distance(1,pca1[0],pca1[1]))
pts.append(Vectors().create_pt_along_vector_at_given_distance(1,pca0[0],pca0[1]))
pts.append(Vectors().create_pt_along_vector_at_given_distance(1,pca2[0],pca2[1]))

ref_pts = []
ref_pts.append([0,0,0])
ref_pts.append([-1,0,0])
ref_pts.append([0,-1,0])
ref_pts.append([0,0,-1])

In [ ]:
landmarks = Landmarks(pts=pts)
icp,txmat = landmarks.align_landmarks(ref_pts)

In [ ]:
aligner = Alignment(lhs=True) 
aligner.transform_folders(output_alignment_using_neuron_axis_and_local_Avg_pia_soma,output_local_rf,icp=icp,txmat = txmat)

In [ ]:
sg = AmiraSpatialGraph(output_alignment_using_neuron_axis_and_local_Avg_pia_soma+'All_Apical_Dendrites_Registered.am')
sg.apply_transformation(txmat)
sg.write_spatial_graph(output_local_rf+'All_Apical_Dendrites_Registered.am')

sg = AmiraSpatialGraph(output_alignment_using_neuron_axis_and_local_Avg_pia_soma+'All_Basal_Dendrites_Registered.am')
sg.apply_transformation(txmat)
sg.write_spatial_graph(output_local_rf+'All_Basal_Dendrites_Registered.am')

sg = AmiraSpatialGraph(output_alignment_using_neuron_axis_and_local_Avg_pia_soma+'All_Neurons_Registered.am')
sg.apply_transformation(txmat)
sg.write_spatial_graph(output_local_rf+'All_Neurons_Registered.am')

In [ ]:
somata = Landmarks()
for file in glob.glob(output_local_rf+'2_Local_Neuron_Axis/*Soma_Centroid.landmarksAscii'):
    somata.append_landmarks(Landmarks(file).pts)
somata.write_landmarks(output_local_rf+'All_Somata_Registered.landmarksAscii')

# Create Rabies Density image

In [ ]:
somata = Landmarks()
for file in glob.glob(input_path_avg_rf+'Local_Ref_Frame_View/*.LandmarkAscii'):
    somata.append_landmarks(Landmarks(file).pts)
somata.write_landmarks(input_path_avg_rf+'Local_Ref_Frame_View/Combined_Rabies_Landmarks.landmarksAscii')

    

In [ ]:
l = Landmarks(input_path_avg_rf+'Local_Ref_Frame_View/Combined_Rabies_Landmarks.landmarksAscii')              
densimage = l.get_density_image(input_path_avg_rf+'Local_Ref_Frame_View/density_image_',voxel_size=400)

In [ ]:
for voxel_size in [400]:#[50,100,200,300,400,450,500]:
    for dens_cutoff_threshold in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
        #for exp_name in exp_names:
        print(voxel_size,dens_cutoff_threshold,)
        l = Landmarks(input_path_avg_rf+'Local_Ref_Frame_View/Combined_Rabies_Landmarks.landmarksAscii')
        
        l.get_density_cluster(output_file=input_path_avg_rf+'Local_Ref_Frame_View/vM1_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                  format(str(voxel_size),str(dens_cutoff_threshold)),\
                                  voxel_size=voxel_size,dens_cutoff_threshold=dens_cutoff_threshold,\
                                  write_density_images=False,min_threshold=0)
        
        
        #print(len(l.pts))
        #l.get_binary_and_contour_images_from_pts(output_file=input_path_avg_rf+'Local_Ref_Frame_View/'+'rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
        #              format(voxel_size,dens_cutoff_threshold),voxel_size=400,cutoff=dens_cutoff_threshold)

In [ ]:
for voxel_size in [400]:#[50,100,200,300,400,450,500]:
    for dens_cutoff_threshold in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
        #for exp_name in exp_names:
        print(voxel_size,dens_cutoff_threshold,)
        l = Landmarks(input_path_avg_rf+'Local_Ref_Frame_View/Density_Clusters/vM1_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii.landmarkAscii'.format(voxel_size,dens_cutoff_threshold))
        
        high_voxels = Landmarks(input_path_avg_rf+'Local_Ref_Frame_View/Density_Clusters/vM1_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii_high_voxel_centers.LandmarkAscii'.format(voxel_size,dens_cutoff_threshold)).pts
        print(len(l.pts))
        l.get_binary_and_contour_images_from_high_voxels(high_voxels,output_file=input_path_avg_rf+'Local_Ref_Frame_View/Density_Clusters/'+'vM1_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii.landmarkAscii'.\
                      format(voxel_size,dens_cutoff_threshold),voxel_size=400,cutoff=dens_cutoff_threshold)

In [ ]:
# Get density cutoff wise landmarks
df = pd.DataFrame(columns=['dens_cutoff','rabies_count','rabies_count_per','morph_count','morph_count_per'])
for dens_cutoff_threshold in [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
    morph_count = len(Landmarks(input_path_avg_rf+'Local_Ref_Frame_View/Density_Clusters/'+'All_Somata_Registered_{}.landmarksAscii.landmarkAscii'.format(dens_cutoff_threshold)).pts)
    rabies_count = len(Landmarks(input_path_avg_rf+'Local_Ref_Frame_View/Density_Clusters/vM1_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii.landmarkAscii'.format(400,dens_cutoff_threshold)).pts)
    
    morph_count_total = len(Landmarks(input_path_avg_rf+'Local_Ref_Frame_View/Density_Clusters/'+'All_Somata_Registered_0.landmarksAscii.landmarkAscii').pts)
    rabies_count_total = len(Landmarks(input_path_avg_rf+'Local_Ref_Frame_View/Density_Clusters/Combined_Rabies_Landmarks.landmarksAscii'.format(400,0)).pts)

    df_tmp = pd.DataFrame()
    df_tmp['dens_cutoff'] = [dens_cutoff_threshold]
    df_tmp['rabies_count'] = [rabies_count]
    df_tmp['morph_count'] = [morph_count]
    df_tmp['rabies_count_per'] = [rabies_count/rabies_count_total]
    df_tmp['morph_count_per'] = [morph_count/morph_count_total]
    
    df = df.append(df_tmp)
df.to_csv(input_path_avg_rf+'Local_Ref_Frame_View/density_clusters.csv')

In [1]:
import pandas as pd
input_path = '/Users/mysee/Documents/PhD/Projects/vM1/Registration_Final_Coronal_Ref_Frame/Input_Ref_Frame/Local_Ref_Frame_View/density_clusters.csv'
df=pd.read_csv(input_path)

In [2]:
df 

,Unnamed: 0,dens_cutoff,morph_count,morph_count_per,rabies_count,rabies_count_per
0,0,0.0,90,1.000000,1567,1.000000
1,0,0.1,76,0.844444,1462,0.932993
2,0,0.2,57,0.633333,1252,0.798979
3,0,0.3,52,0.577778,976,0.622846
4,0,0.4,34,0.377778,804,0.513082
5,0,0.5,24,0.266667,743,0.474154
6,0,0.6,8,0.088889,303,0.193363
7,0,0.7,8,0.088889,148,0.094448
8,0,0.8,8,0.088889,143,0.091257
9,0,0.9,6,0.066667,72,0.045948


# Create layerwise Galleries

## Sort neurons based on depth in local ref frame

In [ ]:
borders,bordes_std

In [ ]:
# Translate neurons so that we can view them in the side view

In [ ]:
l1_apical = AmiraSpatialGraph(generic_graph=True)
l1_basal = AmiraSpatialGraph(generic_graph=True)
l23_apical = AmiraSpatialGraph(generic_graph=True)
l23_basal = AmiraSpatialGraph(generic_graph=True)
l5a_apical = AmiraSpatialGraph(generic_graph=True)
l5a_basal = AmiraSpatialGraph(generic_graph=True)
l5b_apical = AmiraSpatialGraph(generic_graph=True)
l5b_basal = AmiraSpatialGraph(generic_graph=True)
l6_apical = AmiraSpatialGraph(generic_graph=True)
l6_basal = AmiraSpatialGraph(generic_graph=True)

pathlib.Path((output_layerwise_celltypes+'L1/')).mkdir(exist_ok=True)
pathlib.Path((output_layerwise_celltypes+'L23/')).mkdir(exist_ok=True)
pathlib.Path((output_layerwise_celltypes+'L5A/')).mkdir(exist_ok=True)
pathlib.Path((output_layerwise_celltypes+'L5B/')).mkdir(exist_ok=True)
pathlib.Path((output_layerwise_celltypes+'L6/')).mkdir(exist_ok=True)

pathlib.Path((output_local_rf+'For_Visualization/')).mkdir(exist_ok=True)

# read registered cells and find their soma depth... based on the layer border decide their layer cell type
for file in glob.glob(output_local_rf+'2_Local_Neuron_Axis/*soma_wm_axis.am'):
    print(output_local_rf+'1_Surfaces/'+os.path.basename(file)[:-15]+'pia_bottom_open.vtk')
    pia = Surface(input_path_avg_rf+'Local_Ref_Frame_View/Clipped/Avg_lhs_vM1_pia_50.vtk')
    soma_wm_axis = AmiraSpatialGraph(file,generic_graph=True)
    height = Vectors().get_vec_length(soma_wm_axis.graph_data.edge_list[0])
    soma_pia_axis = AmiraSpatialGraph(file[:-10]+'pia_axis.am',generic_graph=True)
    depth = Vectors().get_vec_length(soma_pia_axis.graph_data.edge_list[0])
    depth_per = depth / (depth+height)
    #print(file[:-15]+'apical_dendrite.am')
    if os.path.exists(file[:-15]+'apical_dendrite.am'):
        sg_apical = AmiraSpatialGraph(file[:-15]+'apical_dendrite.am',generic_graph=True)
        sg_basal = AmiraSpatialGraph(file[:-15]+'basal_dendrite.am',generic_graph=True)
        sg_axis = AmiraSpatialGraph(file[:-15]+'soma_pia_axis.am',generic_graph=True)
        
        start_pt = Vectors().create_pt_along_vector_at_given_distance(10000,[0,soma_pia_axis.graph_data.edge_list[0][1][1],soma_pia_axis.graph_data.edge_list[0][1][2]],\
                                                                      [0,soma_pia_axis.graph_data.edge_list[0][0][1],soma_pia_axis.graph_data.edge_list[0][0][2]])
        pia_int_pt,bla = pia.get_vector_intersection_pt(start_pt,\
                                                        [0,soma_pia_axis.graph_data.edge_list[0][1][1],soma_pia_axis.graph_data.edge_list[0][1][2]],\
                                                        extrapolation_len=20000)
        if len(pia_int_pt) > 0:
            new_soma_center = Vectors().create_pt_along_vector_at_given_distance(depth,pia_int_pt,[0,soma_pia_axis.graph_data.edge_list[0][0][1],soma_pia_axis.graph_data.edge_list[0][0][2]])
            new_axis = [new_soma_center,pia_int_pt]
            pts = []
            pts.append(sg_axis.graph_data.edge_list[0][0])
            pts.append(sg_axis.graph_data.edge_list[0][1])
            ref_pts = []
            ref_pts.append(new_axis[0])
            ref_pts.append(new_axis[1])
            Landmarks(pts=pts).write_landmarks(output_root+'pts')
            Landmarks(pts=ref_pts).write_landmarks(output_root+'ref_pts')

            icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
            #aligner = Alignment()
            #aligner.transform_folder(os.path.dirname(file),output_local_rf+'For_Visualization/',os.path.basename(file)[:-15],icp=icp,txmat = txmat,)
            sg_apical.apply_transformation(txmat)
            sg_apical.write_spatial_graph(output_local_rf+'For_Visualization/'+os.path.basename(file)[:-15]+'apical_dendrite.am')
            sg_basal.apply_transformation(txmat)
            sg_basal.write_spatial_graph(output_local_rf+'For_Visualization/'+os.path.basename(file)[:-15]+'basal_dendrite.am')

            if depth_per > 0 and depth_per < borders[0]-bordes_std[0]:
                l1_apical.graph_data = l1_apical.combine_subgraphs([l1_apical.graph_data,sg_apical.graph_data])
                l1_basal.graph_data = l1_basal.combine_subgraphs([l1_basal.graph_data,sg_basal.graph_data])
                sg_apical.write_spatial_graph(output_layerwise_celltypes+'L1/'+os.path.basename(file)[:-15]+'apical_dendrite.am')
                sg_basal.write_spatial_graph(output_layerwise_celltypes+'L1/'+os.path.basename(file)[:-15]+'basal_dendrite.am')

            elif depth_per >= borders[0]-bordes_std[0] and depth_per < borders[1]:
                l23_apical.graph_data = l23_apical.combine_subgraphs([l23_apical.graph_data,sg_apical.graph_data])
                l23_basal.graph_data = l23_basal.combine_subgraphs([l23_basal.graph_data,sg_basal.graph_data])
                sg_apical.write_spatial_graph(output_layerwise_celltypes+'L23/'+os.path.basename(file)[:-15]+'apical_dendrite.am')
                sg_basal.write_spatial_graph(output_layerwise_celltypes+'L23/'+os.path.basename(file)[:-15]+'basal_dendrite.am')

            elif depth_per >= borders[1] and depth_per < borders[2]-bordes_std[2]:
                l5a_apical.graph_data = l5a_apical.combine_subgraphs([l5a_apical.graph_data,sg_apical.graph_data])
                l5a_basal.graph_data = l5a_basal.combine_subgraphs([l5a_basal.graph_data,sg_basal.graph_data])
                sg_apical.write_spatial_graph(output_layerwise_celltypes+'L5A/'+os.path.basename(file)[:-15]+'apical_dendrite.am')
                sg_basal.write_spatial_graph(output_layerwise_celltypes+'L5A/'+os.path.basename(file)[:-15]+'basal_dendrite.am')

            elif depth_per >= borders[2]-bordes_std[2] and depth_per < borders[3]-bordes_std[3]:
                l5b_apical.graph_data = l5b_apical.combine_subgraphs([l5b_apical.graph_data,sg_apical.graph_data])
                l5b_basal.graph_data = l5b_basal.combine_subgraphs([l5b_basal.graph_data,sg_basal.graph_data])
                sg_apical.write_spatial_graph(output_layerwise_celltypes+'L5B/'+os.path.basename(file)[:-15]+'apical_dendrite.am')
                sg_basal.write_spatial_graph(output_layerwise_celltypes+'L5B/'+os.path.basename(file)[:-15]+'basal_dendrite.am')

            else:
                l6_apical.graph_data = l6_apical.combine_subgraphs([l6_apical.graph_data,sg_apical.graph_data])
                l6_basal.graph_data = l6_basal.combine_subgraphs([l6_basal.graph_data,sg_basal.graph_data])
                sg_apical.write_spatial_graph(output_layerwise_celltypes+'L6/'+os.path.basename(file)[:-15]+'apical_dendrite.am')
                sg_basal.write_spatial_graph(output_layerwise_celltypes+'L6/'+os.path.basename(file)[:-15]+'basal_dendrite.am')


l1_apical.write_spatial_graph(output_layerwise_celltypes+'L1_Apical_Dendrite.am')
l23_apical.write_spatial_graph(output_layerwise_celltypes+'L23_Apical_Dendrite.am')
l5a_apical.write_spatial_graph(output_layerwise_celltypes+'L5a_Apical_Dendrite.am')
l5b_apical.write_spatial_graph(output_layerwise_celltypes+'L5b_Apical_Dendrite.am')
l6_apical.write_spatial_graph(output_layerwise_celltypes+'L6_Apical_Dendrite.am')

l1_basal.write_spatial_graph(output_layerwise_celltypes+'L1_Basal_Dendrite.am')
l23_basal.write_spatial_graph(output_layerwise_celltypes+'L23_Basal_Dendrite.am')
l5a_basal.write_spatial_graph(output_layerwise_celltypes+'L5a_Basal_Dendrite.am')
l5b_basal.write_spatial_graph(output_layerwise_celltypes+'L5b_Basal_Dendrite.am')
l6_basal.write_spatial_graph(output_layerwise_celltypes+'L6_Basal_Dendrite.am')

## Make galleries

In [84]:
file = (glob.glob(output_standardization+'Standardized_Uniform_Scaling/Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Pia_Soma_Dist_Preserved/*Neuron*.am'))[0]

In [126]:
output_alignment_final

'/nas1/Data_Mythreya/MotorCortexProject/V11/Registration_Final_Coronal_Ref_Frame/Outputs/4_Final_Registration/'

In [119]:
np.array(borders)*2245,np.array(bordes_std)*2245

(array([ 190.825,  469.205,  877.795, 1748.855]),
 array([ 47.145, 179.6  , 130.21 , 197.56 ]))

In [103]:
pathlib.Path(output_layerwise_celltypes+'Standardized_No_Scaling/').mkdir(exist_ok=True)
pathlib.Path((output_layerwise_celltypes+'Standardized_Uniform_Scaling/')).mkdir(exist_ok=True)
pathlib.Path(output_layerwise_celltypes+'Standardized_No_Scaling/Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Pia_Soma_Dist_Preserved/').mkdir(exist_ok=True)
pathlib.Path((output_layerwise_celltypes+'Standardized_Uniform_Scaling/Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Pia_Soma_Dist_Preserved/')).mkdir(exist_ok=True)
pathlib.Path(output_layerwise_celltypes+'Standardized_No_Scaling/Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Rel_Depth_Dist_Preserved/').mkdir(exist_ok=True)
pathlib.Path((output_layerwise_celltypes+'Standardized_Uniform_Scaling/Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Rel_Depth_Dist_Preserved/')).mkdir(exist_ok=True)

In [137]:
pia = Surface(input_path_avg_rf+'Avg_Pia_lhs_delunay_bottom_open.vtk')
wm = Surface(input_path_avg_rf+'Avg_WM_lhs_delunay_bottom_open.vtk')

for outer_folder in ['Standardized_No_Scaling/','Standardized_Uniform_Scaling/']:
    for inner_folder in ['Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Pia_Soma_Dist_Preserved/','Aligned_Local_Neuron_Axis_To_Ref_Local_Avg_Axis_Rel_Depth_Dist_Preserved/']:
        input_path = output_standardization + outer_folder + inner_folder
        output_path = output_layerwise_celltypes + outer_folder + inner_folder
        
        pathlib.Path(output_path+'L1/').mkdir(exist_ok=True)
        pathlib.Path((output_path+'L23/')).mkdir(exist_ok=True)
        pathlib.Path(output_path+'L5A/').mkdir(exist_ok=True)
        pathlib.Path((output_path+'L5B/')).mkdir(exist_ok=True)
        pathlib.Path(output_path+'L6/').mkdir(exist_ok=True)

        l1_dends = AmiraSpatialGraph(generic_graph=True)
        l23_dends = AmiraSpatialGraph(generic_graph=True)
        l5a_dends = AmiraSpatialGraph(generic_graph=True)
        l5b_dends = AmiraSpatialGraph(generic_graph=True)
        l6_dends = AmiraSpatialGraph(generic_graph=True)
        
        l1_neurons = AmiraSpatialGraph(generic_graph=True)
        l23_neurons = AmiraSpatialGraph(generic_graph=True)
        l5a_neurons = AmiraSpatialGraph(generic_graph=True)
        l5b_neurons = AmiraSpatialGraph(generic_graph=True)
        l6_neurons = AmiraSpatialGraph(generic_graph=True)
        
        i_l1 = 600
        i_l23 = 600
        i_l5a = 600
        i_l5b = 600
        i_l6 = 600
        
        l1_batch = 1
        l23_batch = 1
        l5a_batch = 1
        l5b_batch = 1
        l6_batch = 1
        
        central_axis = AmiraSpatialGraph(output_standardization+'Central_Axis_And_Borders/central_axis.am',generic_graph=True).graph_data.edge_list[0]
        
        for file in sorted(glob.glob(input_path+'*Dend*.am')):
            
            if os.path.basename(file).find('L2')>0 or os.path.basename(file).find('L3')>0 or os.path.basename(file).find('L5')>0 or os.path.basename(file).find('L6')>0: 
                # read the cell find its depthwise type and write to appropriate folder
                sg_dend = AmiraSpatialGraph(file)
                sg_neuron = AmiraSpatialGraph(glob.glob(file[:file.find('Dend')]+'Neuron*.am')[0])

                soma_center = np.array(sg_neuron.neuron.soma.edge_pt_coords).mean(axis=0)
                #print(soma_center)


                soma_pia_axis = AmiraSpatialGraph(output_alignment_final+inner_folder+'2_Local_Neuron_Axis/'+\
                                                    os.path.basename(file)[:os.path.basename(file).find('Dend')]+'soma_pia_axis.am',\
                                                 generic_graph=True).graph_data.edge_list[0]
                #wm_int_pt,bla = wm.get_vector_intersection_pt(soma_pia_axis[1],soma_pia_axis[0],extrapolation_len=20000)
                depth = Vectors().get_vec_length(soma_pia_axis)#abs(soma_center[0][2])

                #depth_per = depth/(Vectors().get_vec_length(soma_pia_axis)+Vectors().get_vec_length([soma_pia_axis[0],wm_int_pt]))
                depth_per = depth / Vectors().get_vec_length(central_axis)
                if depth_per > 0 and depth_per < borders[0]-bordes_std[0]:
                    sg_dend.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i_l1,0,0])
                    sg_neuron.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i_l1,0,0]) 
                    l1_dends.graph_data = l1_dends.combine_subgraphs([l1_dends.graph_data,sg_dend.graph_data])
                    l1_neurons.graph_data = l1_neurons.combine_subgraphs([l1_neurons.graph_data,sg_neuron.graph_data])
                    i_l1 = i_l1 + 600
                    #sg_dend.write_spatial_graph(output_path+'L1/'+os.path.basename(file)[:file.find('Dend')]+'Dend_L1.am')
                    #sg_neuron.write_spatial_graph(output_path+'L1/'+os.path.basename(file)[:file.find('Dend')]+'Neuron_L1.am')
                    if i_l1 > 6000:
                        l1_dends.write_spatial_graph(output_path+'L1_Dend_Gallery_{}.am'.format(l1_batch))
                        l1_dends.write_spatial_graph(output_path+'L1_Neuron_Gallery_{}.am'.format(l1_batch))
                        l1_batch = l1_batch + 1
                        i_l1 = 600
                        l1_dends = AmiraSpatialGraph(generic_graph=True)
                        l1_neurons = AmiraSpatialGraph(generic_graph=True)
                        
                elif depth_per >= borders[0]-bordes_std[0] and depth_per < borders[1]:
                    sg_dend.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i_l23,0,0])
                    sg_neuron.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i_l23,0,0])
                    l23_dends.graph_data = l23_dends.combine_subgraphs([l23_dends.graph_data,sg_dend.graph_data])
                    l23_neurons.graph_data = l23_neurons.combine_subgraphs([l23_neurons.graph_data,sg_neuron.graph_data])
                    #sg_dend.write_spatial_graph(output_path+'L23/'+os.path.basename(file)[:file.find('Dend')]+'Dend_L23.am')
                    #sg_neuron.write_spatial_graph(output_path+'L23/'+os.path.basename(file)[:file.find('Dend')]+'Neuron_L23.am')
                    i_l23 = i_l23 + 600
                    if i_l23 > 6000:
                        l23_dends.write_spatial_graph(output_path+'L23_Dend_Gallery_{}.am'.format(l23_batch))
                        l23_neurons.write_spatial_graph(output_path+'L23_Neuron_Gallery_{}.am'.format(l23_batch))
                        l23_batch = l23_batch + 1
                        i_l23 = 600
                        l23_dends = AmiraSpatialGraph(generic_graph=True)
                        l23_neurons = AmiraSpatialGraph(generic_graph=True)

                elif depth_per >= borders[1] and depth_per < borders[2]:
                    sg_dend.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i_l5a,0,0])
                    sg_neuron.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i_l5a,0,0])
                    l5a_dends.graph_data = l5a_dends.combine_subgraphs([l5a_dends.graph_data,sg_dend.graph_data])
                    l5a_neurons.graph_data = l5a_neurons.combine_subgraphs([l5a_neurons.graph_data,sg_neuron.graph_data])
                    #sg_dend.write_spatial_graph(output_path+'L5A/'+os.path.basename(file)[:file.find('Dend')]+'Dend_L5A.am')
                    #sg_neuron.write_spatial_graph(output_path+'L5A/'+os.path.basename(file)[:file.find('Dend')]+'Neuron_L5A.am')
                    i_l5a = i_l5a + 600
                    if i_l5a > 6000:
                        l5a_dends.write_spatial_graph(output_path+'L5A_Dend_Gallery_{}.am'.format(l5a_batch))
                        l5a_neurons.write_spatial_graph(output_path+'L5A_Neuron_Gallery_{}.am'.format(l5a_batch))
                        l5a_batch = l5a_batch + 1
                        i_l5a = 600
                        l5a_dends = AmiraSpatialGraph(generic_graph=True)
                        l5a_neurons = AmiraSpatialGraph(generic_graph=True)

                elif depth_per >= borders[2] and depth_per < borders[3]-bordes_std[3]:
                    sg_dend.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i_l5b,0,0])
                    sg_neuron.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i_l5b,0,0])
                    l5b_dends.graph_data = l5b_dends.combine_subgraphs([l5b_dends.graph_data,sg_dend.graph_data])
                    l5b_neurons.graph_data = l5b_neurons.combine_subgraphs([l5b_neurons.graph_data,sg_neuron.graph_data])
                    #sg_dend.write_spatial_graph(output_path+'L5B/'+os.path.basename(file)[:file.find('Dend')]+'Dend_L5B.am')
                    #sg_neuron.write_spatial_graph(output_path+'L5B/'+os.path.basename(file)[:file.find('Dend')]+'Neuron_L5B.am')
                    i_l5b = i_l5b + 600
                    if i_l5b > 6000:
                        l5b_dends.write_spatial_graph(output_path+'L5B_Dend_Gallery_{}.am'.format(l5b_batch))
                        l5b_neurons.write_spatial_graph(output_path+'L5B_Neuron_Gallery_{}.am'.format(l5b_batch))
                        l5b_batch = l5b_batch + 1
                        i_l5b = 600
                        l5b_dends = AmiraSpatialGraph(generic_graph=True)
                        l5b_neurons = AmiraSpatialGraph(generic_graph=True)

                else:
                    sg_dend.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i_l6,0,0])
                    sg_neuron.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i_l6,0,0])
                    l6_dends.graph_data = l6_dends.combine_subgraphs([l6_dends.graph_data,sg_dend.graph_data])
                    l6_neurons.graph_data = l6_neurons.combine_subgraphs([l6_neurons.graph_data,sg_neuron.graph_data])
                    #sg_dend.write_spatial_graph(output_path+'L6/'+os.path.basename(file)[:file.find('Dend')]+'Dend_L6.am')
                    #sg_neuron.write_spatial_graph(output_path+'L6/'+os.path.basename(file)[:file.find('Dend')]+'Neuron_L6.am')
                    i_l6 = i_l6 + 600
                    if i_l6 > 6000:
                        l6_dends.write_spatial_graph(output_path+'L6_Dend_Gallery_{}.am'.format(l6_batch))
                        l6_neurons.write_spatial_graph(output_path+'L6_Neuron_Gallery_{}.am'.format(l6_batch))
                        l6_batch = l6_batch + 1
                        i_l6 = 600
                        l6_dends = AmiraSpatialGraph(generic_graph=True)
                        l6_neurons = AmiraSpatialGraph(generic_graph=True)
                        
        l1_dends.write_spatial_graph(output_path+'L1_Dend_Gallery.am')
        l23_dends.write_spatial_graph(output_path+'L23_Dend_Gallery.am')
        l5a_dends.write_spatial_graph(output_path+'L5A_Dend_Gallery.am')
        l5b_dends.write_spatial_graph(output_path+'L5B_Dend_Gallery.am')
        l6_dends.write_spatial_graph(output_path+'L6_Dend_Gallery.am')
        
        l1_neurons.write_spatial_graph(output_path+'L1_Neuron_Gallery.am')
        l23_neurons.write_spatial_graph(output_path+'L23_Neuron_Gallery.am')
        l5a_neurons.write_spatial_graph(output_path+'L5A_Neuron_Gallery.am')
        l5b_neurons.write_spatial_graph(output_path+'L5B_Neuron_Gallery.am')
        l6_neurons.write_spatial_graph(output_path+'L6_Neuron_Gallery.am')
        
        
    

# Evaluate registration

In [12]:
# 1) Diff between original and registered pia depths / wm heights
# 2) Diff between orientation

In [13]:
len(glob.glob(output_neuron_axis+'*soma_wm_axis.am'))

86

In [28]:
df_cols = ['Exp','Original_Axis_Depth','Original_Axis_Height','Registered_Axis_Depth','Registered_Axis_Height',\
           'orientation_original','orientation_reg','orientation_between_original_and_surf_reg',\
           'pia_diff','wm_diff','orientation_difference']
df = pd.DataFrame(columns=df_cols)

# generate a central axis at the somas centroid in the avg rf
pia = Surface(input_path_avg_rf+'Avg_vM1_lhs_Pia.vtk')
wm = Surface(input_path_avg_rf+'Avg_vM1_lhs_WM.vtk')

# read registered cells and find their soma depth... based on the layer border decide their layer cell type
for file in glob.glob(output_neuron_axis+'*soma_wm_axis.am'):
    print(os.path.basename(file)[:-16])
    soma_wm_axis_original = AmiraSpatialGraph(file,generic_graph=True)
    height_o = Vectors().get_vec_length(soma_wm_axis_original.graph_data.edge_list[0])
    soma_pia_axis_original = AmiraSpatialGraph(file[:-10]+'pia_axis.am',generic_graph=True)
    soma_pia_axis_original_local_avg = AmiraSpatialGraph(file[:-10]+'pia_axis_local_avg.am',generic_graph=True)
    #print(soma_pia_axis_original_local_avg.graph_data.edge_list[0],)
    depth_o = Vectors().get_vec_length(soma_pia_axis_original.graph_data.edge_list[0])
    # angle between neuron axis and avg axis
    
    soma_wm_axis_reg = AmiraSpatialGraph(output_alignment_using_neuron_axis_and_local_Avg_pia_soma+'2_Local_Neuron_Axis/'+os.path.basename(file),generic_graph=True)
    height_r = Vectors().get_vec_length(soma_wm_axis_reg.graph_data.edge_list[0])
    soma_pia_axis_reg = AmiraSpatialGraph(output_alignment_using_neuron_axis_and_local_Avg_pia_soma+'2_Local_Neuron_Axis/'+os.path.basename(file)[:-10]+'pia_axis.am',generic_graph=True)
    soma_pia_axis_reg_nearest = AmiraSpatialGraph(output_alignment_using_neuron_axis_and_nearest_pia_soma+'Nearest_Axis/'+os.path.basename(file)[:-15]+'nearest_axis.am',generic_graph=True)
    depth_r = Vectors().get_vec_length(soma_pia_axis_reg.graph_data.edge_list[0])
    #print(height,depth)
    
    # find pia wm intersection pts for the registered neurons
    #start_pt = Vectors().create_pt_along_vector_at_given_distance(10000,soma_pia_axis_reg)
    pia_int_pt,bla = pia.get_vector_intersection_pt(soma_pia_axis_reg.graph_data.edge_list[0][0],soma_pia_axis_reg.graph_data.edge_list[0][1],\
                                                    extrapolation_len=20000)
    wm_int_pt,bla = wm.get_vector_intersection_pt(soma_wm_axis_reg.graph_data.edge_list[0][0],soma_wm_axis_reg.graph_data.edge_list[0][1],\
                                                    extrapolation_len=20000)
    if len(pia_int_pt)==0 or len(wm_int_pt)==0:
        continue
    
    original_vec_len = Vectors().get_vec_length([soma_wm_axis_reg.graph_data.edge_list[0][1],\
                                                 soma_pia_axis_reg.graph_data.edge_list[0][1]])
    new_vec_len = Vectors().get_vec_length([pia_int_pt,wm_int_pt])
    new_depth = (depth_o/original_vec_len)*new_vec_len
    new_soma_location = Vectors().create_pt_along_vector_at_given_distance(new_depth,soma_pia_axis_reg.graph_data.edge_list[0][1],\
                                                                                     soma_pia_axis_reg.graph_data.edge_list[0][0])
    
    depth_r = Vectors().get_vec_length([new_soma_location,pia_int_pt])
    height_r = Vectors().get_vec_length([new_soma_location,wm_int_pt])
    
    df_tmp = pd.DataFrame(columns=df_cols)
    df_tmp['Exp'] = [os.path.basename(file)[:-16]]
    df_tmp['Original_Axis_Depth'] = [depth_o]
    df_tmp['Original_Axis_Height'] = [height_o]
    df_tmp['Registered_Axis_Depth'] = [depth_r]
    df_tmp['Registered_Axis_Height'] = [height_r]
    df_tmp['pia_diff'] = [depth_o-depth_r]
    df_tmp['wm_diff'] = [height_o-height_r]
    
    if len(soma_pia_axis_original_local_avg.graph_data.edge_list) > 0 and len(soma_pia_axis_reg_nearest.graph_data.edge_list) > 0:
        angle_o = Vectors().get_angle_between_vectors(soma_pia_axis_original.graph_data.edge_list[0],\
                                                soma_pia_axis_original_local_avg.graph_data.edge_list[0],ignore_opposite_direction=True)
        angle_r = Vectors().get_angle_between_vectors(soma_pia_axis_reg.graph_data.edge_list[0],\
                                                soma_pia_axis_reg_nearest.graph_data.edge_list[0],ignore_opposite_direction=True)
        print(angle_o,angle_r)
        df_tmp['orientation_original'] = [angle_o]
        df_tmp['orientation_reg'] = [angle_r]
        
        angle = angle_o-angle_r
    else:
        angle = np.nan
        
    df_tmp['orientation_difference'] = [angle_o-angle_r]
    
    soma_pia_axis_surf_reg = AmiraSpatialGraph(output_alignment_using_surfaces+'Using_Surfaces_Pia_Only/2_Local_Neuron_Axis/'+ os.path.basename(file)[:-15]+'soma_pia_axis.am',generic_graph=True)
    
    angle_o_surf_reg = Vectors().get_angle_between_vectors(soma_pia_axis_surf_reg.graph_data.edge_list[0],\
                                                soma_pia_axis_reg.graph_data.edge_list[0],ignore_opposite_direction=True)
    
    df_tmp['orientation_between_original_and_surf_reg'] = [angle_o_surf_reg]
    
    df = df.append(df_tmp)
    
df.to_csv(output_root+'reg_evaluation.csv') 


RA20160322_corrected_shifted_and_merged_DONE
23.234082358864995 29.36649217857826
RA20171212_1_Cell_B.am_Segmented_corrected_DONE
5.991355728122133 2.778291904344825
RA20180108_2.am_Segmented_corrected_shifted_and_merged_DONE
8.537736462515939e-07 13.558235728614676
RA20160329_Cell_A_corrected_shifted_and_merged_DONE
14.537377799398174 33.536834983402166
RA20130701_2_DONE
0.0 26.952868329165938
RA20130628_2_Cell_1_corrected_shifted_and_merged_DONE
18.66022424728887 6.308989508197844
RA20130404_2_corrected_DONE
RA20160401_Cell_B_DONE
1.2074182697257333e-06 10.281504135398219
RA20150806_DONE
0.0 17.07586655944004
RA20130627_DONE
17.838717579678462 23.861989376868877
RA20171214_1_Cell_A.am_Segmented_corrected_shifted_corrected_and_merged_DONE
7.449070339922172 23.640930037259093
RA20130319_2_DONE
21.79594204478988 28.21803694856929
RA20180108_1.am_Segmented_corrected_shifted_and_merged_DONE
nan 19.184008249981726
RA20180109_1_Cell_C_done.am_Segmented_corrected_DONE


/nas1/Data_Mythreya/MotorCortexProject/V11/code/spatial_graphs/Vectors.py:23: RuntimeWarning: invalid value encountered in arccos
  cos_theta = np.abs(np.arccos(np.dot(unit_vec_a,unit_vec_b))*180/np.pi)


nan 23.2051968774824
RA20180412_2_Cell_D.am_Segmented_segmented_DONE
0.0 15.7963435158018
RA20171212_2_Cell_A.am_Segmented_corrected_DONE
20.808957861965787 25.756468778491428
RA20130507_1_corrected_shifted_and_merged_DONE_feature_corrected
16.50966668388871 19.491156130532403
RA20160307_4x_for_merge_.am_corrected.Segmented_DONE
0.0 13.898766627734648
RA20180109_2_Cluster_B_Cell_E_done.am_Segmented_corrected_DONE
8.537736462515939e-07 6.014660312794846
RA20150807_2_Cell_A_corrected_shifted_and_merged_DONE
21.128949475246 30.65039622398944
RA20150810_Cell_A_corrected_shifted_and_merged_DONE
0.0 6.220944944279234
RA20180412_2_Cell_B.am_Segmented_corrected_DONE
8.537736462515939e-07 15.796378435591697
RA20180109_2_Cluster_B_Cell_D_done.am_Segmented_corrected_DONE
8.537736462515939e-07 6.014694573827467
RA20150813_1_Final_Contour_Cell_Merged.am_corrected_Segmented_DONE
18.089972108961888 17.77547400769569
RA20130513_corrected-extended_2corrected_shifted_and_merged_DONE
23.699832286070666 2

In [30]:
import pandas as pd
input_path = '/Users/mysee/Documents/PhD/Projects/vM1/Registration_Final_Coronal_Ref_Frame/Outputs/'
df=pd.read_csv(input_path+'reg_evaluation.csv')
df[(df['pia_diff']<0) & (df['wm_diff']>0)]

,Unnamed: 0,Exp,Original_Axis_Depth,Original_Axis_Height,Registered_Axis_Depth,Registered_Axis_Height,orientation_original,orientation_reg,orientation_between_original_and_surf_reg,pia_diff,wm_diff,orientation_difference


In [5]:
df['orientation_difference'].abs().mean(),df['orientation_difference'].abs().std(),

(9.449686128024664, 6.081788571183838)

In [6]:
df['pia_diff'].abs().mean(),df['pia_diff'].abs().std(),

(100.67508330258245, 90.52664828973512)

In [17]:
len(df[df['pia_diff']>0]['pia_diff'].values),df[df['pia_diff']>0]['pia_diff'].values.mean(),df[df['pia_diff']>0]['pia_diff'].values.std()

(23, 86.70081033955678, 56.026866964527365)

In [18]:
len(df[df['pia_diff']<0]['pia_diff'].values),df[df['pia_diff']<0]['pia_diff'].values.mean(),df[df['pia_diff']<0]['pia_diff'].values.std()

(55, -106.51887017802952, 100.2228233399235)

In [8]:
df['wm_diff'].abs().mean(),df['wm_diff'].abs().std(),

(199.1452248676866, 173.63534341626323)

In [20]:
len(df[df['wm_diff']>0]['wm_diff'].values),df[df['wm_diff']>0]['wm_diff'].values.mean(),df[df['wm_diff']>0]['wm_diff'].values.std()

(23, 146.38934228886797, 115.38852055080432)

In [21]:
len(df[df['wm_diff']<0]['wm_diff'].values),df[df['wm_diff']<0]['wm_diff'].values.mean(),df[df['wm_diff']<0]['wm_diff'].values.std()

(55, -221.20677576428355, 187.05734569322365)